# Raod Map

We want to:

- [x] plot the ACFs for each **ensAvg** **property** individually as a **subplot** in a **space** **figure**
- [ ] make the ACF plotters more efficient.


# General Settings

## Importing packages

In [ ]:
# Importing necessary packages:

from glob import glob
import numpy as np
import pandas as pd

import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage import organizer
from polyphys.analyze import measurer
from polyphys.manage import utilizer

# Project details
project = 'SumRule'
#project = 'TransFoci'
project_details ={
    'SumRule':{
        'parser': SumRule,
        'space_pat': 'N*D*ac*',
        'hierarchy': 'N*',
        'space_hierarchy': 'N*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon', 'dcyl',
                       'dcrowd','phi_c_bulk'
                      ],
        'time_varying_props': [ 'asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'rfloryTMon','shapeTMon', 'transSizeMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['space', 'ensemble_long', 'ensemble', 'nmon',
                             'dcyl','dcrowd', 'phi_c_bulk',
                             'phi_c_bulk_round'
                            ],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'rfloryMon-mean',
                             'rfloryMon-var', 'rfloryMon-sem',
                             'shapeMon-mean', 'shapeMon-var', 'shapeMon-sem',
                             'transSizeMon-mean','transSizeMon-var',
                             'transSizeMon-sem']
    },
    'TransFoci':{
        'parser': TransFoci,
        'space_pat': 'ns*nl*al*D*ac*',
        'hierarchy': 'eps*',
        'space_hierarchy': 'ns*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon_small',
                       'nmon_large','dmon_large', 'dcyl', 'dcrowd',
                       'phi_c_bulk'
                      ],
        'time_varying_props': ['asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'shapeTMon', 'transSizeMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['ensemble_long', 'ensemble', 'space', 'dcyl',
                             'dmon_large', 'nmon_large', 'nmon_small',
                             'dcrowd', 'phi_c_bulk', 'phi_c_bulk_round'],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'shapeMon-mean',
                             'shapeMon-var', 'shapeMon-sem',
                             'transSizeMon-mean','transSizeMon-var',
                             'transSizeMon-sem']
    }
}

properties_labels =  {
    'SumRule': {
        'rfloryMon-norm': {
            'name': 'Flory radius',
            'symbol': r'$\frac{{R_F(\phi_c)}}{{R_{F,0}}}$',
            'color': 'firebrick'
        },
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{{R_g(\phi_c)}}{{R_{g,0}}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{{L(\phi_c)}}{{L_{0}}}$',
            'color': 'forestgreen'
        },
        'transSizeMon-norm': {
            'name': 'Mean radial size',
            'symbol': r'$\frac{{R_{\perp}(\phi_c)}}{{R_{\perp,0}}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{{\Delta(\phi_c)}}{{\Delta_{0}}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{{S(\phi_c)}}{{S_{0}}}$',
            'color': 'orchid'
        },
        'asphericityMon-mean': {
            'name': 'Asphericity',
            'symbol': r'$\Delta(\phi_c)$',
            'color': 'goldenrod'
        },
        'shapeMon-mean': {
            'name': 'Shape parameter',
            'symbol': r'$S(\phi_c)$',
            'color': 'orchid'
        },
        'rPhi': {
            'name': 'Radial volume fraction',
            'symbol': r'$\phi(r)$',
        },
        'zPhi': {
            'name': 'Longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$'
        },
        'rRho': {
            'name': 'Radial number density',
            'symbol': r'$\rho(r)$'
        },
        'zRho': {
            'name': 'Longitudinal number density',
            'symbol': r'$\rho(|z|)$'
        },
        'rPhi-norm': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'$\phi_m(r)$',
            'Crd': r'$\phi_c(r)$',
            'Sum': r'$\sum_i\phi_i(r)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'zPhi-norm': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'$\phi_m(|z|)$',
            'Crd': r'$\phi_c(|z|)$',
            'Sum': r'$\sum_i\phi_i(|z|)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'rPhi-norm-old': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'${{\phi_m(r)}}/{{\phi_m(0)}}$',
            'Crd': r'${{\phi_c(r)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(r)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'zPhi-norm-old': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'${{\phi_m(|z|)}}/{{\phi_m(0)}}$',
            'Crd': r'${{\phi_c(|z|)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(|z|)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'rRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(r)$',
            'Mon': r'${{\rho_m(r)}}/{{\rho_m(0)}}$',
            'Crd': r'${{\rho_c(r)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(r)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        },
        'zRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(|z|)$',
            'Mon': r'${{\rho_m(|z|)}}/{{\rho_m(0)}}$',
            'Crd': r'${{\rho_c(|z|)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(|z|)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        }
    },
    'TransFoci':{
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'transSizeMon-norm': {
            'name': 'Mean radial size',
            'symbol': r'$\frac{{R_{\perp}(\phi_c)}}{{R_{\perp,0}}}$',
            'color': 'forestgreen'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        },
        'asphericityMon-mean': {
            'name': 'Asphericity',
            'symbol': r'$\Delta(\phi_c)$',
            'color': 'goldenrod'
        },
        'shapeMon-mean': {
            'name': 'Shape parameter',
            'symbol': r'$S(\phi_c)$',
            'color': 'orchid'
        },
        'bondsHistFoci-norm': {
            'name': 'Probability distribution function of direct bonds',
            'symbol': r'$\mathcal{P}(x_d)$',
            'color': 'orchid'
        },
        'clustersHistFoci-norm': {
            'name': 'Probability distribution function of clusters',
            'symbol': r'$\mathcal{P}(x_c)$',
            'color': 'orchid'
        },
        'pairDistHistFoci': {
            'name': 'Frequency distribution of pair distance',
            'symbol': r'$\mathcal{H}(n_i,n_j,\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistRdfFoci': {
            'name': 'Radial distribution function of pair distance',
            'symbol': r'$\mathcal{P}(n_i,n_j,\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistRdfGenDistAvg': {
            'name': 'Radial distribution function of pair distance averaged over Genomic distances',
            'symbol': r'$\mathcal{P}(\Delta n;r)$',
            'color': 'orchid'
        },
        'pairDistTFoci': {
            'name': 'Pair distance',
            'symbol': r'$r(t)$',
            'color': 'orchid'
        },
        'zPhi': {
            'name': 'Longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$'
        },
        'rRho': {
            'name': 'Radial number density',
            'symbol': r'$\rho(r)$'
        },
        'zRho': {
            'name': 'Longitudinal number density',
            'symbol': r'$\rho(|z|)$'
        },
        'rPhi-norm': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'$\phi_m(r)$',
            'Foci': r'$\phi_M(r)$',
            'Crd': r'$\phi_c(r)$',
            'Sum': r'$\sum_i\phi_i(r)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'zPhi-norm': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'$\phi_m(|z|)$',
            'Foci': r'$\phi_M(|z|)$',
            'Crd': r'$\phi_c(|z|)$',
            'Sum': r'$\sum_i\phi_i(|z|)/a_i$',
            'Sum_constant': r'$\sum_c\phi_c(\infty)/a_c$'
        },
        'rPhi-norm-old': {
            'name': 'Normalized radial volume fraction',
            'symbol': r'$\phi(r)$',
            'Mon': r'${{\phi_m(r)}}/{{\phi_m(0)}}$',
            'Foci': r'${{\phi_M(r)}}/{{\phi_M(0)}}$',
            'Crd': r'${{\phi_c(r)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(r)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'zPhi-norm-old': {
            'name': 'Normalized longitudinal volume fraction',
            'symbol': r'$\phi(|z|)$',
            'Mon': r'${{\phi_m(|z|)}}/{{\phi_m(0)}}$',
            'Foci': r'${{\phi_M(|z|)}}/{{\phi_M(0)}}$',
            'Crd': r'${{\phi_c(|z|)}}/{{\phi_c(\infty)}}$',
            'Sum': r'${(\sum_i\phi_i(|z|)/a_i)}/{(\sum_i\phi_i(\infty)/a_i)}$',
            'Sum_constant': r'${(\sum_c\phi_c(\infty)/a_c)}/{(\sum_i\phi_i(\infty)/a_i)}$'
        },
        'rRho-norm': {
            'name': 'Normalized radial number density',
            'symbol': r'$\rho(r)$',
            'Mon': r'${{\rho_m(r)}}/{{\rho_m(0)}}$',
            'Foci': r'${{\rho_M(r)}}/{{\rho_M(0)}}$',
            'Crd': r'${{\rho_c(r)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(r)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        },
        'zRho-norm': {
            'name': 'Normalized longitudinal number density',
            'symbol': r'$\rho(|z|)$',
            'Mon': r'${{\rho_m(|z|)}}/{{\rho_m(0)}}$',
            'Foci': r'${{\rho_M(|z|)}}/{{\rho_M(0)}}$',
            'Crd': r'${{\rho_c(|z|)}}/{{\rho_c(\infty)}}$',
            'Sum': r'${(\sum_i\rho_i(|z|)a_i^2)}/{(\sum_i\rho_i(\infty)a_i^2)}$',
            'Sum_constant': r'${(\sum_c\rho_c(\infty)a_c^2)}//{(\sum_i\rho_i(\infty)a_i^2)}$'
        }
    }
}
attr_labels = {
    "phi_c_bulk_round": "$\phi_c$",
    "time": "$\hat{{t}}$",
    "time_norm": "$\hat{{t}}/\hat{{t}}^{{max}}$",
    "lags": "$\hat{t}_{lags}$",
    "dmon_large": "${{a_M}}/{{a_m}}$",
    "dcrowd": "${{a_c}}/{{a_m}}$",
    "dcyl": "${{D}}/{{a_m}}$",
    "nmon_small": "$N_m$",
    "nmon_large": "$N_M$",
    'phi_c_bulk_norm': "${a\phi_c}/{a_c}$",
    'genomic_distance':  '$\Delta n$',
    'bin_center':'$r$',
    'bin_center-norm-dmon_large':'${{r}}/{{a_M}}$',
    'bin_center-norm':'${{r}}/{{r_{max}}}$',
    'bin_center-r':'$r$',
    'bin_center-norm-r':'${{2r}}/{{D}}$',
    'bin_center-recentered-norm-r':'${{(2r-a^{shift})}}/{{D}}$',
    'bin_center-dcrowd-r':'${{2r}}/{{a_c}}$',
    'bin_center-dcrowd-recentered-r':'${{(2r-a^{shift})}}/{{a_c}}$',
    'bin_center-z':'$z$',
    'bin_center-norm-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-recentered-z':'${{|z|}}/{{z_{max}}}$',
    'bin_center-theta':'$\theta$',
    'bin_center-dcrowd-theta':'${{\theta}}/{{\pi}}$',
    'bin_center-norm-theta':'${{\theta}}/{{\pi}}$',
    'nmon':'$N$',
    'nmon_large':'$N_M$',
    'phi_c_rescaled':'$\\frac{{a\phi_c}}{{a_c}}$',
    'confinement_rate':'$\kappa=\\frac{{a_c}}{{D-a_c}}$',
    'confinement_rate_r':'$\kappa=\\frac{{D-a_c}}{{a_c}}$',
    'dep_energy_max':'$\mathcal{F}_{dep}=\phi_c^{(bulk)}[1+{{3a_m}}{{2a_c}}]$',
    'int_energy_max':'$\mathcal{F}_{int}=\\frac{{Na\phi_c^{(bulk)}}}{{a_c}}[3a_ma_c + \\frac{{3}}{{2}}]$',
    'species': 'Type'
}
dmon_large_c_palette = ['orange', 'darkgoldenrod', 'forestgreen']

In [ ]:
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = tuner.truncated_colormap(flar_cmap,  min_value=0., max_value=1, ncolors=1000)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)

In [ ]:
# loading databases:
#analysis_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets'
analysis_db = '/Users/amirhsi_mini/research_data/analysis/'
#analysis_db = '../../Datasets/' # relative
#project = 'TransFoci'
phase='ensAvg'
geometry='biaxial'
group = "bug"
species = "Mon"
space_dbs = glob(analysis_db + project_details[project]['space_pat'])
print(space_dbs)

# Auto-correlation functions

##### Note: In a given space, ensAvg groups differs in their values of bulk volume fraction of crowders (column name: phi_c_bulk)

### Theoretical background:

Below, all the several definitions of autocorrelation function (acf) in the literature are listed. Assuming $A(t)$ is a discrete time-varying property of interest and is collected every $M$ timesteps in a molecular dynamics simulation with the time step $\Delta t$ and time unit $\hat{t}$. First, we define the mean (or average) and unbiased variance of $A(t)$ as what follows

$$\langle A\rangle=\frac{1}{t_{max}}\sum_{t=1}^{t_{max}}A(t)$$
$$\sigma^2_A=\langle A^2 \rangle-\langle A\rangle^2=\frac{1}{t_{max}-1}\sum_{t=1}^{t_{max}}(A(t)-\langle A\rangle)^2$$

where $t_{max}$ is the total number of collected cofigurations or timesteps, or simply the size of $A(t)$, implying the total number of simulated timesteps is $t_{max}\times M$. Given this definition, the acf has been defined in the following ways in the literature:

1. Allen and Tildesley define the *non-normalized* acf in "Computer simulation of liquids - 2017" as

$$c_{AA}(h) = \frac{1}{h_{max}}\sum_{h_0=1}^{h_{max}}A(h_0)A(h_0+h)$$

where $h_{max}=\frac{t_{max}-1}{h}$

2. *1989 - Murat M Greet GS - Structure of a Grafted Polymer Brush A Molecular Dynamics Simulation* defines the acf in "Structure of a Grafted Polymer Brush A Molecular Dynamics Simulation - 1989" as

$$c_{AA}(h) =\frac{\langle (A(h)-\langle A \rangle)(A(0)-\langle A \rangle)\rangle}{\sigma^2_A}=\frac{\langle A(h)A(0)\rangle-\langle A \rangle \langle A(0) \rangle}{\sigma^2_A}$$

See the rest of this artciel to see how correlation time is calculated. See the rest of this artciel to see how correlation time is calculated.

3. In *1989 - Grest GS Kremer K Witten TA et el - Relaxation of Self-Entangled Many-Arm Star Polymers, 1987 - Grest GS Kremer K Witten TA - Structure of Many-Arm Star Polymers A Molecular Dynamics Simulation, and 1977 - Kranbuehl DE Verdier PH - Relaxation of the aspherical shapes of random-coil polymer chains*, the acf is

$$c_{AA}(h) =\frac{\langle A(h)A(0)\rangle-\langle A \rangle^2}{\sigma^2_A}$$

where the average is performed over the initial time steps $h=h_0=0$ taken every $K$ timesteps. Kranbuehl and Verdier runs the whole simulation several times the relaxation time of the end-to-end length.

4. In *1978 - Rapaport DC - Molecular dynamics simulation of polymer chains with excluded volum, and 1979 - Bishop M Ceperley D Frisch HL - Molecular dynamics of polymeric systems* the afc is given by

$$c_{AA}(h) =\frac{\langle A(h)A(h_0+h)\rangle-\langle A(h_0) \rangle^2}{\sigma^2_A(h_0)}$$

where

$$\sigma^2_A(h_0)=\langle A(h_0)^2 \rangle-\langle A(h_0)\rangle^2$$

The averaging should be done over ensembles; however, in practice, the ensemble average is replaced by the time average provided that the timesteps or time interval between successvie $h_0$ values are sufficently lagre so the configurations are uncorrelated.

5. *1981 - Bruns W Bansal R - Molecular dynamics study of a single polymer chain in solution* defines the afc as

$$c_{\vec{A}\vec{A}}(h) = \frac{\sum_{h_0=1}^{t_{max}-h}\vec{A}(h_0).\vec{A}(h_0+h)}{\sum_{h_0=1}^{t_{max}-h}\vec{A}^2(h_0)}$$

where $\vec{A}$ is a vector quantity and $h_0$ is summed over $1,1+p,1+2p,\dots$ instead of $h_0=1,2,3,\dots,t_{max}-h$. For a scalar quantity, this definition is used:

$$c_{AA}(h) = \frac{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A(h_0)A(h_0+h)-\sum_{h_0=1}^{t_{max}-h}A(h_0)\sum_{h_0=1}^{t_{max}-h}A(h_0+h)}{\sqrt{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A^2(h_0)-[\sum_{h_0=1}^{t_{max}-h}A^2(h_0)]^2}-\sqrt{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A^2(h_0+h)-[\sum_{h_0=1}^{t_{max}-h}A^2(h_0+h)]^2}}=
\frac{\langle A(h_0)A(h_0+h)\rangle-\langle A(h_0)\rangle\langle A(h_0+h)\rangle}{\sqrt{\langle A^2(h_0)\rangle-\langle A(h_0)\rangle^2}-\sqrt{\langle A^2(h_0+h)\rangle-\langle A(h_0+h)\rangle^2}}$$

6. *Brockwell PJ Davis RA - 2016 - Introduction to Time Series and Forecasting* defines the afc in the following way

$$c_{AA}(h) = \frac{cov_{A}(h)}{\sigma_A^2} = \frac{1}{\sigma_A^2}\frac{1}{t_{max}}\sum_{h_0=1}^{t_{max}-h} [A(h_0+h)-\langle A\rangle][A(h_0)-\langle A\rangle]$$





### How to run this Notebook:

1. Check ((spaces_to_read**
2. Check **group** and **species**.
3. check **database**.
4. delete extra **physical** properties based on the preoject.
5. check **nlags** and **ticks**.
6. check **parser** and **spaces_title**.


### Grouping and filtering

Since the datasets we work with are large, it is a good idea to
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
acf_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('acf.parquet.brotli')
                ]
acf_space_dbs

In [ ]:
# read data
acf_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('acf.parquet.brotli')
                ]
acf = [pd.read_parquet(space_db) for space_db in acf_space_dbs]
acf = pd.concat(acf,axis=0, ignore_index=True)
acf.reset_index(inplace=True,drop=True)

# a sorted list of unique spaces in the dataset
spaces = acf.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)

# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
acf['phi_c_bulk_round'] = acf['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
acf = acf.loc[~acf['phi_c_bulk_round'].isin([0.025, 0.05, 0.075, 0.125, 0.175]),:]
acf.reset_index(inplace=True, drop=True)
# setting colors for unique crd_c
phi_crds = acf.loc[:,'phi_c_bulk_round'].unique()
phi_crds.sort()
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = tuner.truncated_colormap(flar_cmap,  min_value=0.0, max_value=1.0, ncolors=200)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)
phi_colors = sns.color_palette("flare_cmap_cut", len(phi_crds))#,as_cmap=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
properties = [property_ for property_ in acf.columns if '-mean' in property_]
properties.sort()
# Define unique specifications for each physical property:
acf_props_specs =  {
    'SumRule': {
        'rfloryTMon': {
            'name': 'Flory radius',
            'symbol': r'$C_{R_FR_F}(\hat{t}_{lags})$',
            'color': 'firebrick'
        },
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$C_{R_gR_g}(\hat{t}_{lags})$',
            'color':'steelblue'
                   },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$C_{LL,||}(\hat{t}_{lags})$',
            'color': 'forestgreen'
        },
        'transSizeTMon': {
            'name': 'transverse furthermost distance',
            'symbol': r'$C_{LL,\perp}(\hat{t}_{lags})$',
            'color': 'orange'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$C_{\Delta\Delta}(\hat{t}_{lags})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$C_{SS}(\hat{t}_{lags})$',
            'color': 'orchid'
        }
    },
    'TransFoci': {
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$C_{R_gR_g}(\hat{t}_{lags})$',
            'color':'steelblue'
                   },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$C_{LL,||}(\hat{t}_{lags})$',
            'color': 'forestgreen'
        },
        'transSizeTMon': {
            'name': 'transverse furthermost distance',
            'symbol': r'$C_{LL,\perp}(\hat{t}_{lags})$',
            'color': 'orange'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$C_{\Delta\Delta}(\hat{t}_{lags})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$C_{SS}(\hat{t}_{lags})$',
            'color': 'orchid'
        }
    }
}
acf_props_specs_project = acf_props_specs[project]
space_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    space_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        #'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    space_titles[space] = space_title_style[project]

### Some explorations:

In [ ]:
acf.head()

In [ ]:
acf.columns

In [ ]:
acf.info()

### ACF with CIs for all the physical properties per space columns wrapped with $\phi_c^{bulk}$

In [ ]:
nlags=7000
ticks = int(nlags/7)
for space in spaces:
    space_acf = acf[acf.space==space]
    plotter.p_acf_with_ci_space(
        space_acf,
        space,
        space_titles[space],
        project,
        properties=acf_props_specs_project,
        ncols=3,
        xlimits=(0, nlags, ticks),
        ylimits=(-0.25, 1, 0.25),
        lags=nlags,
        dpi=100,
        fontsize=18
    )

### ACF of each physical property in a project collored with an attribute

In [ ]:
#property_= 'gyrTMon'
legend_anchor = (1.1,1.02)
nrows = len(spaces)
for property_ in acf_props_specs_project.keys():
    plotter.p_acf_allInOne_project(
        acf,
        project,
        space_titles,
        property_,
        acf_props_specs_project[property_],
        phi_crds,
        phi_colors,
        xlimits=(0, nlags, ticks),
        nrows=nrows,
        ncols=1,
        legend_anchor=legend_anchor,
        lags=nlags
    )

# Chain size plots

### Grouping and filtering

Since the datasets we work with are large, it is a good idea to
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
chainsize_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('chainSize.parquet.brotli')
                      ]
chainsize = [pd.read_parquet(space_db) for space_db in chainsize_space_dbs]
chainsize = pd.concat(chainsize,axis=0, ignore_index=True)
chainsize.reset_index(inplace=True,drop=True)
print(chainsize.columns)
# droping some of the columns
cols_to_drop = {
    'SumRule': ['asphericityTMon-var', 'asphericityTMon-sem', 'fsdTMon-var',
                'fsdTMon-sem', 'gyrTMon-var', 'gyrTMon-sem', 'rfloryTMon-var',
                'rfloryTMon-sem', 'shapeTMon-var', 'shapeTMon-sem', 'transSizeTMon-var', 'transSizeTMon-sem'],
    'TransFoci': ['asphericityTMon-var', 'asphericityTMon-sem', 'fsdTMon-var',
                  'fsdTMon-sem', 'gyrTMon-var', 'gyrTMon-sem',
                  'shapeTMon-var','shapeTMon-sem','transSizeTMon-var', 'transSizeTMon-sem']
}
chainsize.drop(columns=cols_to_drop[project],inplace=True)
# a sorted list of unique spaces in the dataset
spaces = chainsize.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
chainsize['phi_c_bulk_round'] = chainsize['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize = chainsize.loc[~chainsize['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize.reset_index(inplace=True, drop=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
ext = '-mean'
properties = [
    prop.split(ext)[0] for prop in chainsize.columns if prop.endswith(ext)
]
properties.sort()
# change the name of cols after fining properties based on "mean" in their name
new_columns = [prop.split(ext)[0] for prop in chainsize.columns]
chainsize.columns = new_columns
# Define unique specifications for each physical property:


In [ ]:
properties

In [ ]:
chainsize_props_labels =  {
    'SumRule': {
        'rfloryTMon': {
            'name': 'Flory radius',
            'symbol': r'$R_F(\hat{t})$',
            'color': 'firebrick'
        },
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$R_g(\hat{t})$',
            'color':'steelblue'
        },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L_{||}(\hat{t})$',
            'color': 'forestgreen'
        },
        'transSizeTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L_{\perp}(\hat{t})$',
            'color': 'crismon'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$\Delta(\hat{t})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$S(\hat{t})$',
            'color': 'orchid'
        }
    },
    'TransFoci':{
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$R_g(\hat{t})$',
            'color':'steelblue'
        },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L_{||}(\hat{t})$',
            'color': 'forestgreen'
        },
        'transSizeTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L_{\perp}(\hat{t})$',
            'color': 'crismon'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$\Delta(\hat{t})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$S(\hat{t})$',
            'color': 'orchid'
        }
    }
}

In [ ]:
space_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    space_title_style = {
        'SumRule': fr"$N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        #'TransFoci': fr"$n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    space_titles[space] = space_title_style[project]

### Some explorations:

In [ ]:
chainsize.head()

In [ ]:
chainsize.columns

In [ ]:
chainsize.info()

### Time series per physical properties per space with an attribute for hues and cols

In [ ]:
ext = 'pdf'
save_to = './'
hue_attr = 'phi_c_bulk_round'
col_attr = 'phi_c_bulk_round'
x_prop = 'time'
rc_params = {
    'mathtext.default': 'regular',
    'text.usetex': True
}
golden_ratio = 1.618
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
col_wrap = 3
fig_title_kws = {'x': 0.5, 'y': 1.0}
for space in spaces:
    chainsize_space = chainsize.loc[chainsize['space']==space]
    for y_prop in properties:
        sns.set_theme(
        context='talk',
        style='ticks',
        font='Times New Roman',
        font_scale=2,
        rc=rc_params
        )
        tseries_grid = sns.relplot(
                data=chainsize_space,
                x=x_prop,
                y=y_prop,
                hue=hue_attr,
                col=col_attr,
                kind='line',
                aspect=golden_ratio,
                facet_kws=facet_kws,
                legend='full',
                ci=None,
                col_wrap=col_wrap
            )
        tseries_grid.set_axis_labels(
            attr_labels[x_prop], 
            chainsize_props_labels[project][y_prop]['symbol']
        )
        tseries_grid.set_titles(
            attr_labels[col_attr]+r"$={col_name}$"
        )
        tseries_grid.tight_layout(w_pad=1)
        tseries_grid._legend.set_title(attr_labels[hue_attr])
        tseries_grid.fig.suptitle(space_titles[space], **fig_title_kws)
        output = "-".join(["tseries", project, space, y_prop]) + "." + ext
        tseries_grid.savefig(save_to + output, bbox_inches='tight')
        plt.close()

### Time series of each property per porject colored by space col wrapped by attribute

#### SumRule

In [ ]:
max_time_per_space = chainsize.groupby('ensemble_long')['time'].max()
max_time_per_space = max_time_per_space.reset_index()

In [ ]:
#chainsize['time_norm'] = 0
for ens_long in max_time_per_space['ensemble_long']:
    max_time = \
        max_time_per_space.loc[
            max_time_per_space['ensemble_long']==ens_long, 'time'].values[0]
    ens_cond = chainsize['ensemble_long'] == ens_long
    chainsize.loc[ens_cond, 'time_norm'] = \
        chainsize.loc[ens_cond, 'time'] / max_time

In [ ]:
phi_c_to_keep = [0, 0.1, 0.2, 0.3, 0.4]
chainsize_phi_c_filtered = \
    chainsize.loc[chainsize['phi_c_bulk_round'].isin(phi_c_to_keep),:]
chainsize_phi_c_filtered.reset_index(inplace=True)

In [ ]:
# Below definition depends on how organize input paramaters in a project
ext ='pdf'
save_to = './'
font_scale = 2
rc_params = {
    'mathtext.default': 'regular',
    'text.usetex': True
}
x_prop = 'time_norm'
col_attr = 'nmon'
col_order = chainsize_phi_c_filtered[col_attr].unique()
col_order.sort()
row_attr = 'phi_c_bulk_round'
row_order = chainsize_phi_c_filtered[row_attr].unique()
row_order.sort()
hue_attr = 'dcrowd'
style_attr = 'dcyl'
style_order = chainsize_phi_c_filtered[style_attr].unique()
style_order.sort()
height = 6
aspect = 1.618 # golden ratio
color_palette='rocket_r'
fig_title_kws = {'x': 0.5, 'y': 1.0}
for y_prop in properties:
    sns.set_theme(
        context='talk',
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
    tseries_grid = sns.relplot(
        data=chainsize_phi_c_filtered,
        x=x_prop,
        y=y_prop,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        style=style_attr,
        style_order=style_order,
        hue=hue_attr,
        height=height,
        aspect=aspect,
        alpha=0.8,
        legend='full',
        kind='line',
        ci=None,
        facet_kws=facet_kws,
    )
    tseries_grid.set_axis_labels(
        attr_labels[x_prop],
        chainsize_props_labels[project][y_prop]['symbol']
    )
    tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name} $ | " + attr_labels[col_attr] + r"$={col_name}$")
    tseries_grid.legend.set_title(attr_labels[hue_attr])
    tseries_grid.tight_layout(w_pad=1)
    output = "-".join(["tseries", project, y_prop, hue_attr, col_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

#### TransFoci

In [ ]:
x_prop = 'time'
font_scale = 2

# Below definition depends on how organize input paramaters in a project
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

ext ='pdf'
save_to = './'

col_attr = 'phi_c_bulk_round'
hue_attr = 'dmon_large'
col_wrap = 3
height = 5
aspect = 1.5 * 1.618 # golden ratio
color_palette='colorblind'
fig_title_kw = {'x': 0.5, 'y': 0.98}
for y_prop in properties:
    tseries_grid = plotter.p_tseries_allInOne_space(
        chainsize,
        project_titles[project],
        x_prop,
        y_prop,
        hue_attr,
        col_attr,
        chainsize_props_labels[project],
        attr_labels,
        plot_context='talk',
        fig_title_kw=fig_title_kw,
        font_scale=font_scale,
        height=height,
        aspect=aspect,
        col_wrap=col_wrap,
        color_palette=color_palette
    )
    output = "-".join(["tseries", project, y_prop, hue_attr, col_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

col_attr = 'dmon_large'
hue_attr = 'phi_c_bulk_round'
col_wrap = 1
height = 5
aspect = 2 * 1.618 # golden ratio
color_palette='rocket_r'
fig_title_kw = {'x': 0.5, 'y': 0.96}
for y_prop in properties:
    tseries_grid = plotter.p_tseries_allInOne_space(
        chainsize,
        project_titles[project],
        x_prop,
        y_prop,
        hue_attr,
        col_attr,
        chainsize_props_specs_project,
        attr_labels,
        fig_title_kw=fig_title_kw,
        font_scale=font_scale,
        height=height,
        aspect=aspect,
        col_wrap=col_wrap,
        color_palette=color_palette
    )

# Equilbrium size-related measures

## Equilibrium chain-size measures

#### SumRule

In [ ]:
#species = 'Mon'
#allInOne_db = '../../Datasets/' # mac-mini
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'bug'
geometry = 'biaxial'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
attributes = project_details[project]['equil_attributes']
#norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
norm_props = [
    'gyrMon-norm', 'fsdMon-norm', 'rfloryMon-norm', 'asphericityMon-norm',
    'shapeMon-norm', 'transSizeMon-norm'
]
#norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'rfloryMon-norm', 'asphericityMon-mean', #'shapeMon-mean','transSizeMon-norm'
#]
selected_cols = attributes + norm_props
value_vars = norm_props
#gyr_fsd_only_norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
#]
#selected_cols = attributes + gyr_fsd_only_norm_props
#value_vars = gyr_fsd_only_norm_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)
chainsize_melted['confinement_rate'] = np.round(
        (chainsize_melted['dcrowd'] / 
         (chainsize_melted['dcyl'] - chainsize_melted['dcrowd'])), 3
)

##### S and Delta not nomed

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

row_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

style_attr = 'nmon'
style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'dcyl'
size_order = sorted(chainsize_melted[size_attr].unique())
sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcrowd'
hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=False,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
#    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.2,0.2,0.2,0.4,0.4]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, 0.2, 1)
    ymax = utilizer.round_up_nearest(ymax, 0.2, 1)
    print(properties_labels[project][prop]["symbol"])
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.2*ystep, ymax+0.2*ystep)
    ax.set_title("")
#tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = [attr_labels[hue_attr]] + list(hue_order)  + \
    [attr_labels[size_attr]] + list(size_order) +  \
        [attr_labels[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

##### S and Delta nomred

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

row_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

style_attr = 'nmon'
style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'dcyl'
size_order = sorted(chainsize_melted[size_attr].unique())
sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcrowd'
hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=False,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
#    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.2,0.2,0.2,0.2]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, 0.2, 1)
    ymax = utilizer.round_up_nearest(ymax, 0.2, 1)
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.2*ystep, ymax+0.2*ystep)
    ax.set_title("")
#tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = [attr_labels[hue_attr]] + list(hue_order)  + \
    [attr_labels[size_attr]] + list(size_order) +  \
        [attr_labels[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

##### a_c=a

In [ ]:
dcrowd_cond = (chainsize_melted['dcrowd'].isin([1.0]))
cond = dcrowd_cond
data = chainsize_melted.loc[cond, :].copy()
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

col_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

#style_attr = 'dcyl'
#style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'nmon'
size_order = sorted(data[size_attr].unique())
sizes=[3+2*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcyl'
hue_order = sorted(data[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 2
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    #style=style_attr,
    #style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=True,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=10,
    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.1,0.2,0.02,0.05]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, ystep, 2)
    ymax = utilizer.round_up_nearest(ymax, ystep, 1)
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.5*ystep, ymax+0.5*ystep)
    ax.set_title("")
#tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = ([attr_labels[hue_attr]] + list(hue_order)  + \
    [attr_labels[size_attr]] + list(size_order)# +  \
        #[attr_labels[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures-DcrowdEqualDmon'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

#### TransFoci

In [ ]:
#species = 'Mon'
#allInOne_db = '../../Datasets/' # mac-mini
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'bug'
geometry = 'biaxial'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
attributes = project_details[project]['equil_attributes']
#norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
norm_props = [
    'gyrMon-norm', 'fsdMon-norm', 'asphericityMon-norm', 'shapeMon-norm'
]
selected_cols = attributes + norm_props
value_vars = norm_props
#gyr_fsd_only_norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
#]
#selected_cols = attributes + gyr_fsd_only_norm_props
#value_vars = gyr_fsd_only_norm_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)
# dictionary of normalizer properties' features
project_titles = {}
spaces = chainsize_equil['space'].unique()
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        #'SumRule': fr" $N={s_info.nmon}, {{D}}/{{a}}={s_info.dcyl}, {{a_c}}/{{a}}={s_info.dcrowd}$",
        'TransFoci': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'
hue_attr = 'dmon_large'
col_attr = 'variable'
col_order = value_vars
col_wrap = 2
save_to = './'
ext = 'pdf'
# The three lines below are for transc foci
color_palette = dmon_large_c_palette
markers=['o', 'o', '^']
dashes=[(2, 2), (2, 2), (1, 0)]
#color_palette = 'colorblind'
plot_context = 'talk'
font_scale = 2
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
facet_kws = {
        'sharey': False,
        'sharex': True,
        'legend_out': True
        }
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'bbox_to_anchor': (0.12, 0.52),
    'frameon': True,
    'facecolor': 'lightgray',
    'borderpad': 0.1
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    legend='full',
    hue=hue_attr,
    style=hue_attr,
    kind='line',
    dashes=dashes,
    markers=markers,
    height=height,
    aspect=aspect,
    palette=color_palette,
    col_wrap=col_wrap,
    facet_kws=facet_kws,
    markersize=15
)
for ax, prop in zip(tseries_grid.axes.flat, col_order):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title(None)
axis_ls = tseries_grid.axes[1]
vline_patch = tuner.marker_ls_handler(
    ['False', 'True'], ['--', '-'], ['o', '^'],color='black'
)
axis_ls.legend(
    title='Collapse transition',
    handles=vline_patch,
    bbox_to_anchor=(0.64, 0.53),
    frameon=True,
    facecolor='lightgray',
    borderpad=0.1
)
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.legend.set_title(attr_labels[hue_attr])
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
#tseries_grid.tight_layout(w_pad=1)
tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(yticks=np.arange(0,1.2,0.2), xticks=np.arange(0,0.5,0.1))
output = "-".join(["equilPlot", project, 'chainMeasures', x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## Clusters and Bonds of Large monomers (foci)

### Merging bonds and cluster data

In [ ]:
hist_properties = ['clustersHistFoci', 'bondsHistFoci']
#hist_names = dict(zip(hist_properties, ['clusters', 'bonds']))
measures = ['mean', 'sem', 'var']
species = 'Foci'
bin_center_labels = {
   'clustersHistFoci': '$x_c$',
   'bondsHistFoci': '$x_d$' 
}
save_to = './'
ext = 'pdf'
foci = []
for prop in hist_properties:
    cols_to_drop = [prop + '-' + measure for measure in  measures]
    attr_labels['bin_center'] = bin_center_labels[prop]
    hist_df = '-'.join(['allInOne', project, group, prop + '.parquet.brotli'])
    hist_df = analysis_db + hist_df
    hist_df = pd.read_parquet(hist_df)
    hist_df.drop(columns=cols_to_drop, inplace=True)
    #hist_df.reset_index(inplace=True, drop=True)
    hist_df['property'] = prop
    hist_df.rename(columns={prop + '-norm': 'value'}, inplace=True)
    foci.append(hist_df)
foci =  pd.concat(foci, axis=0)
spaces = foci[['nmon_small','nmon_large', 'dcyl', 'dcrowd']].drop_duplicates().sort_values(by=['nmon_small','nmon_large', 'dcyl', 'dcrowd']).values
project_titles = {}
for (nmon_small, nmon_large, dcyl, dcrowd) in spaces:
    project_title_style = {
        #'SumRule': fr" $N={s_info.nmon}, {{D}}/{{a_s}}={s_info.dcyl}, {{a_c}}/{{a_s}}={s_info.dcrowd}$",
        'TransFoci': fr" $N_m={nmon_small}, N_M={nmon_large}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

### Bonds and cluster: phi_c as col, dmon_large as hue

In [ ]:
phi_c_bulk_cond = (foci['phi_c_bulk_round'].isin([0.1, 0.2, 0.3, 0.4]))
conds = phi_c_bulk_cond
foci_filtered = foci.loc[conds,:]
foci_filtered.reset_index(drop=True, inplace=True)
nmon_large = foci_filtered['nmon_large'].unique()[0]
data = foci_filtered
x_prop = 'bin_center'
y_prop = 'value'
col_attr = 'property'
col_order = sorted(data[col_attr].unique())
row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())
hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette

save_to = './'
ext = 'pdf'
rc_params = {
'axes.facecolor': 'aliceblue',
'mathtext.default': 'regular',
'text.usetex': True,
'axes.grid': True,
'axes.grid.axis': 'both',
#'axes.grid.which': 'both'
}
plot_context = 'talk'
# hue-col pair specific settings
height = 6
aspect = 1.618
font_scale = 2
legend_style = 'full'
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
fig_title_kws = {'x': 0.45, 'y': 0.97, 'ha': 'center'}
move_legend_kws = {
    'ncol': 1,
    #'bbox_to_anchor': (0.19, 0.66),  # col_wrap = 4
    #'bbox_to_anchor': (0.23, 0.74), # col_wrap = 3
    'bbox_to_anchor': (0.3, 0.9),  # col_wrap = 2
    'frameon': True,
    'facecolor': 'lightgray'
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.catplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    ls="-",
    height=height,
    alpha=0.7,
    aspect=aspect,
    palette=color_palette,
    legend=legend_style,
    sharey=False,
    sharex=False,
    kind='point'
)
tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$")
row_col_pairs = list(itertools.product(row_order,col_order))
for idx, (ax, (row_name, col_name)) in enumerate(zip(tseries_grid.axes.flat,row_col_pairs)):
    ax.set_ylabel(properties_labels[project][col_name+'-norm']['symbol'])
    if idx >= (len(row_col_pairs) - len(col_order)):
        ax.set_xlabel(bin_center_labels[col_name])
    else:
        ax.axes.xaxis.set_ticklabels([])    
tseries_grid.set(yticks=np.arange(0,1.2,0.2))
sns.move_legend(tseries_grid, title=attr_labels[hue_attr], loc='upper left', **move_legend_kws)
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
#tseries_grid.set(yticks=np.arange(0, 1.2, 0.2))
output = "-".join(['equilPlot', project, y_prop, x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

### Bonds and cluster: phi_c as hue, dmon_large as col

In [ ]:
#phi_c_bulk_cond = (foci['phi_c_bulk_round'].isin([0.1, 0.2, 0.3, 0.4]))
#conds = phi_c_bulk_cond
foci_filtered = foci#.loc[conds,:]
foci_filtered.reset_index(drop=True, inplace=True)
data = foci
x_prop = 'bin_center'
y_prop = 'value'
col_attr = 'property'
col_order = sorted(data[col_attr].unique())
row_attr = 'dmon_large'
row_order = sorted(data[row_attr].unique())
hue_attr = 'phi_c_bulk_round'
hue_order = sorted(data[hue_attr].unique())
color_palette = 'rocket_r'
#color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
save_to = './'
ext = 'pdf'
rc_params = {
'axes.facecolor': 'aliceblue',
'mathtext.default': 'regular',
'text.usetex': True,
'axes.grid': True,
'axes.grid.axis': 'both',
'axes.grid.which': 'both'
}
plot_context = 'talk'
height = 6
aspect = 1.618
font_scale = 2
legend_style = 'full'
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
fig_title_kws = {'x': 0.45, 'y': 0.95, 'ha': 'center'}
move_legend_kws = {
    'ncol': 1,
    #'bbox_to_anchor': (0.19, 0.66),  # col_wrap = 4
    'bbox_to_anchor': (0.81, 0.89), # col_wrap = 3
    #'bbox_to_anchor': (0.3, 0.9),  # col_wrap = 2
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.catplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    ls="-",
    height=height,
    alpha=0.7,
    aspect=aspect,
    palette=color_palette,
    legend=legend_style,
    sharey=False,
    sharex=False,
    kind='point'
)
tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$")
row_col_pairs = list(itertools.product(row_order,col_order))
xlims = {
    'bondsHistFoci': [0, nmon_large-1],
    'clustersHistFoci': [1, nmon_large]
}
for idx, (ax, (row_name, col_name)) in enumerate(zip(tseries_grid.axes.flat,row_col_pairs)):
    #ax.set_xlim(xlims[col_name])
    #ax.set_ylim(xlims[col_name])
    ax.set_ylabel(properties_labels[project][col_name+'-norm']['symbol'])
    if idx >= (len(row_col_pairs) - len(col_order)):
        ax.set_xlabel(bin_center_labels[col_name])
    else:
        ax.axes.xaxis.set_ticklabels([])       
tseries_grid.set(yticks=np.arange(0,1.2,0.2))
#tseries_grid.legend.set_title(attr_labels[hue_attr])
sns.move_legend(tseries_grid, title=attr_labels[hue_attr], loc='upper left', **move_legend_kws)
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
output = "-".join(['equilPlot', project, y_prop, x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## Pair Distance Stats

### Importing dataset

In [ ]:
pair_dist_properties = ['pairDistHistFoci', 'pairDistRdfFoci']
prop = pair_dist_properties[1]
# LOad data
dist_map = '-'.join(['allInOne', project, group, 'pairDistStats.parquet.brotli'])
dist_map = analysis_db + dist_map
dist_map = pd.read_parquet(dist_map)
# a sorted list of unique spaces in the dataset
spaces = dist_map['space'].unique()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        'SumRule': fr" $N={s_info.nmon}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$",
        'TransFoci': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
dist_map['phi_c_bulk_round'] = dist_map['phi_c_bulk'].apply(
        utilizer.round_up_nearest, args=[divisor, round_to]
        )
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
dist_map = dist_map.loc[~dist_map['phi_c_bulk_round'].isin(phi_c_to_drop),:]
dist_map.reset_index(inplace=True, drop=True)
# Dropping pair distance cols of `prop_to_drop`
prop_to_drop = pair_dist_properties[0]
cols_to_drop = [col for col in dist_map.columns if col.startswith(prop_to_drop)]
dist_map.drop(columns=cols_to_drop, inplace=True)
dist_map['bin_center-norm-dmon_large'] = \
    dist_map['bin_center'] / dist_map['dmon_large']
# renaming pair distance cols of `prop`
pair_tags = {
    col: col.split('-')[1] for col in dist_map.columns if 'genDist' in col
    }
dist_map.rename(columns=pair_tags, inplace=True)
# Now, just keep the new col names for melting
pair_tags = list(pair_tags.values())
pair_names = {tag: organizer.sort_by_alphanumeric(tag)[1::2] for tag in pair_tags}
pair_names = {tag: 
    r"$({0},{1},{2})$".format(int(locs[0]), int(locs[1]), int(locs[2])) for tag, locs in pair_names.items()}
attributes = project_details[project]['equil_attributes'].copy()
attributes.extend(['bin_center', 'bin_center-norm', 'bin_center-norm-dmon_large'])
sel_cols = attributes + pair_tags
pair_dist = dist_map[sel_cols].melt(
    id_vars=attributes,
    value_vars=pair_tags,
    var_name=prop
)
pair_dist.loc[:, 'genomic_distance'] = pair_dist.loc[
    :, 'pairDistRdfFoci'].str.split(r'(\d+\.*\d*)', regex=True).apply(
    lambda str_list: int(float(str_list[-2])))

### Genomic position distance per d_mon

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (pair_dist['bin_center-norm']<=0.3)
dmon_large_cond = (pair_dist['dmon_large'].isin([1,5]))
phi_c_cond = (pair_dist['phi_c_bulk_round'].isin([0, 0.1, 0.2]))
conds = bin_center_cond & dmon_large_cond & phi_c_cond
data = pair_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm-dmon_large'
y_prop = 'value'

col_attr = 'phi_c_bulk_round'
col_order = sorted(data[col_attr].unique())
row_attr = 'dmon_large'
row_order = sorted(data[row_attr].unique())
style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

hue_attr = 'pairDistRdfFoci'
hue_order = sorted(data[hue_attr].unique(),key=organizer.sort_by_alphanumeric)
hue_labels = [pair_names[hue] for hue in hue_order]
color_palette = 'colorblind'


height = 7
aspect = 1.618
plot_context = 'talk'

rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.82, 0.89), # col_wrap = 3
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    style=style_attr,
    style_order=style_order,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    legend='full',
    alpha=0.7,
    lw=4,
    ci=None,
    )
legend_labels = ['$(n_1, n_2,\Delta n)$'] + hue_labels +  [attr_labels[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(8)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
for idx, dmon_large in enumerate(row_order):
    for ax in tseries_grid.axes[idx,:]:
        ax.axvline(1.0, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        vline_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=4)
        ax.legend(
            handles=vline_patch,
            frameon=True,
            facecolor='lightgray'
        )
tseries_grid.set_titles(
    attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicPos']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

### Genomic distance matters not genomic position

In [ ]:
bin_center_cond = (pair_dist['bin_center']<=20)
dmon_large_chosen =[5]
dmon_large_cond = (pair_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.15, 0.3]
phi_c_cond = (pair_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = bin_center_cond & dmon_large_cond & phi_c_cond
data = pair_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm-dmon_large'
y_prop = 'value'

hue_attr = 'pairDistRdfFoci'
hue_order = sorted(data[hue_attr].unique(),key=organizer.sort_by_alphanumeric)
hue_labels = [pair_names[hue] for hue in hue_order]

color_palette = 'colorblind'

col_attr = 'phi_c_bulk_round'
col_order = sorted(data[col_attr].unique())

row_attr = 'genomic_distance'
row_order = sorted(data[row_attr].unique())

height = 8
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.79, 0.85), # col_wrap = 3
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    row=row_attr,
    row_order=row_order,
    col=col_attr,
    col_order=col_order,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    legend='full',
    alpha=0.7,
    ls='--',
    lw=5,
    ci=None,
    )
legend_labels = ['$(n_1, n_2,\Delta n)$'] + hue_labels
for idx, new_label in enumerate(legend_labels[1:]):
    tseries_grid._legend.legendHandles[idx].set_linewidth(8)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

for idx, dmon_large in enumerate(row_order):
    for ax in tseries_grid.axes[idx,:]:
        ax.axvline(1.0, label=r"$r=a_M$", lw=5, ls=':', color='black', alpha=0.8, zorder=1)
        vline_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=4)
        ax.legend(
            handles=vline_patch,
            frameon=True,
            facecolor='lightgray'
        )
tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
#tseries_grid.set(xlim=(0,10), ylim=[0,0.75])
tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(
    project_titles[project] + f", ${{a_M}}/{{a_m}}={dmon_large_chosen[0]}$", **fig_title_kws)
sns.move_legend(tseries_grid, title=legend_labels[0], loc='upper left', **move_legend_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'matters']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

### Genomic-distance-averaged data

##### Generating dataset

In [ ]:
group_by_cols = pair_dist.columns.to_list()
group_by_cols.remove('value')
group_by_cols.remove('pairDistRdfFoci')
agg_dict = ['last'] * len(group_by_cols)
agg_dict = dict(zip(group_by_cols,agg_dict))
agg_dict['value'] = 'mean'
# averaging over genomic distance:
gen_dist = pair_dist.groupby(group_by_cols).agg(agg_dict)
gen_dist.reset_index(inplace=True, drop=True)
gen_dist.rename(columns={'value': 'pairDistRdfGenDistAvg'}, inplace=True)

##### Genomic distance explained

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.4)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.1, 0.3]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
ext = 'pdf'
save_to = './'
x_prop = 'bin_center-norm'

y_prop = 'pairDistRdfGenDistAvg'

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

col_attr = 'dmon_large'
col_order = sorted(data[col_attr].unique())

style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette
#color_palette = 'colorblind'

height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.45, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 2,
    #'bbox_to_anchor': (0.66, 0.31),
    'bbox_to_anchor': (0.26, 0.71),
    'frameon': True,
    'facecolor': 'lightgray'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    style=style_attr,
    style_order=style_order,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    ci=None,
    lw=4
    )
legend_labels = [attr_labels[hue_attr]] + hue_order + ['$\Delta n$'] + style_order
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

tseries_grid.axes[0,0].legend(
    handles=tseries_grid._legend.legendHandles[:len(hue_order)+1],
    labels=legend_labels[:len(hue_order)+1],
    frameon=True,
    facecolor='lightgray',
    loc='upper right'
)
tseries_grid.axes[1,0].legend(
    handles=tseries_grid._legend.legendHandles[len(hue_order)+1:],
    labels=legend_labels[len(hue_order)+1:],
    frameon=True,
    facecolor='lightgray'
)
#legend_labels = [attr_labels[hue_attr]] + list(col_order) +  [attr_labels[size_attr]] + list(size_order)
#for idx, new_label in enumerate(legend_labels):
 #   tseries_grid._legend.texts[idx].set_text(new_label)

for idx, row in enumerate(row_order):
    data_row = gen_dist.loc[gen_dist[row_attr]==row,:]
    for idx_col, (ax, col) in enumerate(zip(tseries_grid.axes[idx,:], col_order)):
        bin_center_max = data_row.loc[data_row[col_attr]==col, 'bin_center'].max()
        y_max = data_row.loc[data_row[col_attr]==col, y_prop].max()
        dcrowd = data_row.loc[data_row[col_attr]==col, 'dcrowd'].unique()[0] # dcorwd is fixed here, so we use unique
        ax.set_ylim(0-0.04*y_max,y_max+0.04*y_max)
        ax.axvline(col / bin_center_max, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        x_range = np.linspace(0, (col+dcrowd)/bin_center_max,100)
        ax.fill_between(
            x_range, 1.05*y_max, 0,color='darkorchid', alpha=0.2, zorder=0, label='Direct contact'
        )
        fill_color_patch = tuner.color_handler(
            ['Direct contact'], ['darkorchid'], alpha=0.2
        )
        vl_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=6)
        if (idx == 2) & (idx_col==0):
            ax.legend(handles=fill_color_patch+vl_patch,
                      frameon=True,
                      facecolor='lightgray',
                      loc='upper right',
                      #bbox_to_anchor=(0.565, 0.18)
                     )
tseries_grid.set_ylabels(properties_labels[project][y_prop]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'explained']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

###### Line size not line style

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.4)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.1, 0.3]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
ext = 'pdf'
save_to = './'
x_prop = 'bin_center-norm'

y_prop = 'pairDistRdfGenDistAvg'

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

col_attr = 'dmon_large'
col_order = sorted(data[col_attr].unique())

style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

size_attr = 'genomic_distance'
size_order = sorted(data[size_attr].unique())
sizes=[3+2*i for i in range(len(size_order))]

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette
#color_palette = 'colorblind'

height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.45, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 2,
    #'bbox_to_anchor': (0.66, 0.31),
    'bbox_to_anchor': (0.26, 0.71),
    'frameon': True,
    'facecolor': 'lightgray',
    'markerscale': 4
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    #style=style_attr,
    #style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    #dashes=True,
    #markers=True,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    ci=None,
    markersize=15,
    markevery=5,
    ls="--"
    )
legend_labels = [attr_labels[hue_attr]] + hue_order + ['$\Delta n$'] + size_order
for idx, new_label in enumerate(legend_labels):
    #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

tseries_grid.axes[0,0].legend(
    handles=tseries_grid._legend.legendHandles[:len(hue_order)+1],
    labels=legend_labels[:len(hue_order)+1],
    frameon=True,
    facecolor='lightgray',
    loc='upper right'
)
tseries_grid.axes[1,0].legend(
    handles=tseries_grid._legend.legendHandles[len(hue_order)+1:],
    labels=legend_labels[len(hue_order)+1:],
    frameon=True,
    facecolor='lightgray',
    markerscale=2
)
#legend_labels = [attr_labels[hue_attr]] + list(col_order) +  [attr_labels[size_attr]] + list(size_order)
#for idx, new_label in enumerate(legend_labels):
 #   tseries_grid._legend.texts[idx].set_text(new_label)

for idx, row in enumerate(row_order):
    data_row = gen_dist.loc[gen_dist[row_attr]==row,:]
    for idx_col, (ax, col) in enumerate(zip(tseries_grid.axes[idx,:], col_order)):
        bin_center_max = data_row.loc[data_row[col_attr]==col, 'bin_center'].max()
        ax.axvline(col / bin_center_max, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        vl_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=6)
        if (idx == 2) & (idx_col==0):
            ax.legend(handles=vl_patch,
                      frameon=True,
                      facecolor='lightgray',
                      #loc='lower left',
                      #bbox_to_anchor=(0.565, 0.18)
                     )
tseries_grid.set_ylabels(properties_labels[project][y_prop]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.set_titles(attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'explained']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

##### Genomic distance as col - noting important

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.3)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0.05, 0.2, 0.3,]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

x_prop = 'bin_center-norm'
y_prop = 'pairDistRdfGenDistAvg'

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette

col_attr = 'genomic_distance'
col_order = sorted(data[col_attr].unique())

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

ext = 'pdf'
save_to = './'
alpha = 0.7
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.92, 0.5)
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=alpha,
    ci=None,
    )
#for idx, new_label in enumerate(legend_labels[1:]):
#    tseries_grid._legend.texts[idx].set_text(new_label)
#tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.set_titles(
    attr_labels[row_attr] + r"$={row_name}$" +
    " | " + attr_labels[col_attr] + r"$={col_name}$"
    )
tseries_grid.legend.set_title(legend_labels[0])
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=1)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, hue_attr, col_attr, row_attr, 'colStyle']
    )
tseries_grid.savefig(save_to + output + "-LessInfo." + ext, bbox_inches='tight')
plt.close()

### Pair Dist time-series per project: Not working

In [ ]:
prop = 'pairDistTFoci'
# Load data:
dist_tseries = '-'.join(['allInOne', project, group, 'pairDistT.parquet.brotli'])
dist_tseries = analysis_db + dist_tseries
dist_tseries = pd.read_parquet(dist_tseries)
# a sorted list of unique spaces in the dataset
spaces = dist_tseries['space'].unique()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
dist_tseries['phi_c_bulk_round'] = dist_tseries['phi_c_bulk'].apply(
        utilizer.round_up_nearest, args=[divisor, round_to]
        )
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
dist_tseries = dist_tseries.loc[~dist_tseries['phi_c_bulk_round'].isin(phi_c_to_drop),:]
dist_tseries.reset_index(inplace=True, drop=True)
# renaming pair distance cols of `prop`
pair_tags = {
    col: col.split('-')[1] for col in dist_tseries.columns if 'genDist' in col
    }
dist_tseries.rename(columns=pair_tags, inplace=True)
# Now, just keep the new col names for melting
pair_tags = list(pair_tags.values())
pair_names = [organizer.sort_by_alphanumeric(tag)[1::2] for tag in pair_tags]
pair_names = [
    r"$({0},{1},{2})$".format(int(locs[0]), int(locs[1]), int(locs[2])) for locs in pair_names]

In [ ]:
attributes = ['nmon_large','dmon_large', 'phi_c_bulk_round', 'time']
sel_cols = attributes + pair_tags
dist_tseries_melted = dist_tseries[sel_cols].melt(
    id_vars=attributes,
    value_vars=pair_tags,
    var_name=prop
)

In [ ]:
phi_c_chosen = [0., 0.2, 0.25, 0.3, 0.35, 0.4]
dist_tseries_melted_by_phi_c = dist_tseries_melted.loc[dist_tseries_melted['phi_c_bulk_round'].isin(phi_c_chosen),:]

legend_labels = ['$(n_1, n_2,\Delta n)$'] + pair_names
x_prop = 'time'
y_prop = 'value'
hue_attr = 'pairDistTFoci'
col_attr = 'dmon_large'
row_attr = 'phi_c_bulk_round'
color_palette = 'colorblind'
alpha = 0.7
height = 6
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.91, 0.5)
}

plotter.p_pairDist_allInOne_project_colStyle(
    dist_tseries_melted_by_phi_c,
    project,
    project_title_style[project],
    x_prop,
    y_prop,
    hue_attr,
    col_attr,
    row_attr,
    project_details[project],
    attr_labels,
    legend_labels,
    height=height,
    aspect=aspect,  # golden ratio
    color_palette=color_palette,
    plot_context=plot_context,
    rc_params=rc_params,
    font_scale=font_scale,
    save_to=save_to,
    facet_kws=facet_kws,
    alpha=alpha,
    move_legend_kws=move_legend_kws
)

In [ ]:
phi_c_chosen = [0.25, 0.275, 0.3, 0.325, 0.35]
phi_c_chosen = [0.25, 0.3, 0.35]
phi_cond = (dist_tseries_melted['phi_c_bulk_round'].isin(phi_c_chosen))
dmon_large_chosen = 5.0
dmon_larg_cond = (dist_tseries_melted['dmon_large']==dmon_large_chosen)
dist_tseries_melted_by_phi_c_dmon_large = dist_tseries_melted.loc[phi_cond & dmon_larg_cond,:]

legend_labels = ['$(n_1, n_2,\Delta n)$'] + pair_names
x_prop = 'time'
y_prop = 'value'
col_attr = 'pairDistTFoci'
color_palette = 'Set2'
hue_attr = 'phi_c_bulk_round'
alpha = 0.7
height = 6
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.91, 0.5)
}
ext = 'pdf'
col_wrap=3
save_to = './'
axes_style = 'ticks'
font_family = 'Times New Roman'
fig_title_kw = {'x': 0.5, 'y': 1.0}
sns.set_theme(
        context=plot_context,
        style=axes_style,
        palette=color_palette,
        font=font_family,
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=dist_tseries_melted_by_phi_c_dmon_large,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_wrap=col_wrap,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=alpha,
    ci=None
)
for ax, pair_name in zip(tseries_grid.axes.flat, pair_names):
        ax.set_title('$(n_1, n_2,\Delta n)=$' + pair_name)
tseries_grid.set_ylabels(norm_props_specs[project][col_attr]['symbol'])
tseries_grid.set_xlabels(attr_labels[x_prop])
tseries_grid.legend.set_title(attr_labels[hue_attr])
sns.move_legend(tseries_grid, "upper left", bbox_to_anchor=(.35, .25), frameon=False)
tseries_grid.fig.suptitle(project_title_style[project], **fig_title_kw)
tseries_grid.tight_layout(w_pad=0)
output = "-".join(
    ["equilPlot", col_attr, hue_attr, 'al'+str(dmon_large_chosen), 'phic' + str(len(phi_c_chosen))]
)
tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
plt.close()

## Local distributions: Different plotting styles for different projects (not working)

### Sum-rule  project

In [ ]:
dir_prop_pairs

In [ ]:
directions = ['z', 'r']
props = ['Phi', 'Rho']
dir_prop_pairs = list(itertools.product(directions, props))

species_list = ['Mon', 'Crd']
species_names = ['Monomers', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))

allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'all'
geometry = 'biaxial'
for direction, prop in dir_prop_pairs[:1]:
    prop_fullname = direction + prop
    equil_db = '-'.join(
        ['allInOne', project, group, direction + 'LocalDist.parquet.brotli']
    )
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # dropping extra columns
    redundants = [
        'HistCrd-mean', 'HistCrd-var', 'HistCrd-sem', 'HistCrd-norm',
        'HistMon-mean', 'HistMon-var', 'HistMon-sem', 'HistMon-norm',
        'PhiCrd-var', 'PhiCrd-sem', 'PhiMon-var', 'PhiMon-sem', 
        'RhoCrd-var','RhoCrd-sem', 'RhoMon-var', 'RhoMon-sem'
    ]
    cols_to_drop = [direction + col for col in redundants]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # dropping crowders distributions with phi_c=0
    local_phi_c_cond = \
        (local_dist['phi_c_bulk_round'] == 0) & (local_dist[direction + prop +'Crd-mean'] == 0)
    local_dist = local_dist.loc[~local_phi_c_cond,:]
    # melting based on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes = attributes + ['bin_center', 'bin_center-norm']
    hist_cols = list(set(local_dist.columns).difference(set(attributes)))
    hist_cols_norm = [col for col in hist_cols if col.endswith('norm')]
    prop_cols_norm = [col for col in hist_cols_norm if prop in col]
    sel_cols_norm = attributes + prop_cols_norm
    local_prop_norm = local_dist[sel_cols_norm].melt(
        id_vars=attributes,
        value_vars=prop_cols_norm,
        var_name=prop_fullname
    )
    # prop_fullname
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
    )
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: species_names_dict[x]
    )
    # plotting style
    ext = 'pdf'
    save_to = './'

    data = local_prop_norm

    x_prop = 'bin_center-norm'
    x_prop_name = 'bin_center-norm' + '-' + direction
    y_prop = 'value'
    
    style_attr = 'dcyl'
    style_order = data[style_attr].unique()
    style_order.sort()
    
    hue_attr = 'phi_c_bulk_round'
    hue_order = data[hue_attr].unique()
    hue_order.sort()
    
    row_attr = 'dcrowd'
    row_order = data[row_attr].unique()
    row_order.sort()
    
    col_attr = prop_fullname
    col_order = species_names
    
    size_attr = 'nmon'
    size_order = data[size_attr].unique()
    size_order.sort()

    color_palette = 'rocket_r'
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
            'mathtext.default': 'regular',
            'text.usetex': True
        }
    save_to = './'
    font_scale = 2.5
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
    move_legend_kws = {'bbox_to_anchor': (0.8, 1.0)}
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        marker=True,
        markevery=7,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels =  [attr_labels[hue_attr]] + list(hue_order) +  [attr_labels[size_attr]] + list(size_order) + [attr_labels[style_attr]] + list(style_order)
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_titles(
        "{col_name} | " + attr_labels[row_attr] + r"$={row_name}$"
    )
    tseries_grid.tight_layout(w_pad=1)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(["equilPlot", project, prop_fullname, 'LocalDist'])
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

### Trans-Foci project

In [ ]:
directions = ['z', 'r']
props = ['Phi', 'Rho']
species_list = ['Foci', 'Mon', 'Crd']
species_names = ['Big monomers', 'Small monomers', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
dir_prop_pairs = list(itertools.product(directions, props))
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'all'
geometry = 'biaxial'
for direction, prop in dir_prop_pairs:
    prop_fullname = direction + prop
    equil_db = '-'.join(
        ['allInOne', project, group, direction + 'LocalDist.parquet.brotli']
    )
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # dropping extra columns
    redundants = [
        'HistCrd-mean', 'HistCrd-var', 'HistCrd-sem', 'HistCrd-norm',
        'HistMon-mean', 'HistMon-var', 'HistMon-sem', 'HistMon-norm',
        'PhiCrd-var', 'PhiCrd-sem', 'PhiMon-var', 'PhiMon-sem', 
        'RhoCrd-var','RhoCrd-sem', 'RhoMon-var', 'RhoMon-sem'
    ]
    cols_to_drop = [direction + col for col in redundants]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # dropping crowders distributions with phi_c=0
    local_phi_c_cond = \
        (local_dist['phi_c_bulk_round'] == 0) & (local_dist[direction + prop +'Crd-mean'] == 0)
    local_dist = local_dist.loc[~local_phi_c_cond,:]
    # melting based on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes = attributes + ['bin_center', 'bin_center-norm']
    hist_cols = list(set(local_dist.columns).difference(set(attributes)))
    hist_cols_norm = [col for col in hist_cols if col.endswith('norm')]
    prop_cols_norm = [col for col in hist_cols_norm if prop in col]
    sel_cols_norm = attributes + prop_cols_norm
    local_prop_norm = local_dist[sel_cols_norm].melt(
        id_vars=attributes,
        value_vars=prop_cols_norm,
        var_name=prop_fullname
    )
    # prop_fullname
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
    )
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: species_names_dict[x]
    )
    # plotting style
    #species_order = [prop_fullname + species + '-norm' for species in species_list]
    ext = 'pdf'
    save_to = './'

    data = local_prop_norm
    
    species_legend = ['Particle type'] + species_names
    
    x_prop = 'bin_center-norm'
    x_prop_name = 'bin_center-norm' + '-' + direction
    y_prop = 'value'
    
    
    hue_attr = 'phi_c_bulk_round'
    hue_order = data[hue_attr].unique()
    hue_order.sort()
    
    col_attr = 'dmon_large'
    col_order = data[col_attr].unique()
    col_order.sort()
    
    row_attr = prop_fullname
    row_order = species_names
    
    color_palette = 'rocket_r'
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
            'mathtext.default': 'regular',
            'text.usetex': True
        }
    save_to = './'
    font_scale = 2.5
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
    move_legend_kws = {'bbox_to_anchor': (0.85, 0.9)}
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels = [attr_labels[hue_attr]] + list(hue_order) + species_legend
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_titles("{row_name} | " + attr_labels[col_attr] + r"$={col_name}$")
    tseries_grid.tight_layout(w_pad=1)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(["equilPlot", project, prop_fullname, 'LocalDist'])
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

## Sum Rule equation: Different plotting styles for different projects

### Sum-rule  project

##### Meging different distributions along different directions

In [ ]:
directions = ['r','z']
props = ['Phi', 'Rho']
dir_prop_pairs = list(itertools.product(directions, props))
species_list = ['Mon', 'Crd']
normalizing_methods = ['norm', 'scale-normalized_curve']
# 'norm': normalizing each curve with its deeip inside species-rich region
# 'scale-normalized_curve': normalizing each curve to make the area-under-cruve 1.
species_names = ['Chain', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
species_names_dict['Sum'] = 'Sum'
# col to drops for each prop
measures = ['mean','sem', 'var']
ens_avg_measures = [
    species + '-' + col for species in species_list for col in measures
]
# Database location
#allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
allInOne_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets/'
group = 'all'
geometry = 'biaxial'
var_cols = []
dist = []
for normalizing_method in normalizing_methods:
    dist_per_norm = []
    melting_dict = [species + '-' + normalizing_method for species in species_list]
    melting_dict.append('Sum-sumrule_local')
    for direction, prop in dir_prop_pairs:
        prop_fullname = direction + prop
        equil_db = '-'.join(['allInOne', project, group, prop_fullname])
        equil_db += '-NormalizedScaled.parquet.brotli'
        equil_db = allInOne_db + equil_db
        local_dist = pd.read_parquet(equil_db)
        # Dropping redundant columns
        cols_to_drop = [prop_fullname + redund for redund in ens_avg_measures]
        local_dist.drop(columns=cols_to_drop, inplace=True)
        # Sumrule at local level
        local_dist[prop_fullname+'Sum-sumrule_local'] = 0
        local_dist[prop_fullname+'Sum-sumrule_constant'] = 0
        for species in species_list:
            local_dist[prop_fullname+'Sum-sumrule_local'] += \
                local_dist[prop_fullname+species+'-scale']
        local_dist[prop_fullname+'Sum-sumrule_local'] = \
            (local_dist[prop_fullname+'Sum-sumrule_local'] /
             local_dist[prop_fullname+'Crd-normalizer'])
        # Melting based local distributions on species:
        attributes = project_details[project]['equil_attributes'].copy()
        attributes += ['bin_center', 'bin_center-norm', 'bin_center-dcrowd', 'bin_center-dcrowd-recentered']
        var_name =  'species'
        var_cols = [prop_fullname + col for col in melting_dict]
        sel_cols = attributes + var_cols
        local_dist_melted = local_dist[sel_cols].melt(
            id_vars=attributes,
            value_vars=var_cols,
            value_name=normalizing_method+'_'+var_name+'_value',
            var_name=var_name
        )
        # Renaming values
        local_dist_melted.loc[:, var_name] = \
            local_dist_melted.loc[:, var_name].apply(
                lambda x: x.split('-')[0].split(prop_fullname)[-1]
            )
        local_dist_melted.loc[:, var_name] = \
            local_dist_melted.loc[:, var_name].apply(
                lambda x: species_names_dict[x]
        )
        # Dropping distributions with phi_c=0
        local_dist_melted = \
            local_dist_melted[local_dist_melted['phi_c_bulk_round']!=0]
        # Fixing near-end problem in crowder distributions in z directio
        if direction == 'z':
            cond = local_dist_melted['bin_center-norm'] <= 0.5
            local_dist_melted = local_dist_melted.loc[cond, :]
        # local_dist_melted
        local_dist_melted['direction'] = direction
        local_dist_melted['property'] = prop
        dist_per_norm.append(local_dist_melted)
    dist_per_norm = pd.concat(dist_per_norm, axis=0)
    dist_per_norm.reset_index(inplace=True, drop=True)
    dist_per_norm.drop(columns=['ensemble', 'space', 'ensemble_long'], inplace=True)
    dist.append(dist_per_norm)
dist = pd.concat(dist, axis=1)
dist.reset_index(inplace=True, drop=True)
dist = dist.loc[:, ~dist.columns.duplicated()].copy()
del directions, props, dir_prop_pairs, species_list, normalizing_methods
del normalizing_method, melting_dict, species_names, species_names_dict
del measures, ens_avg_measures, allInOne_db, group, geometry, var_cols, prop
del direction, local_dist, local_dist_melted, var_name,  attributes, sel_cols

#### Sum rule alone

In [ ]:
# Filtering dataset
for direction in ['z','r']:
    property_ = 'Phi'
    var_col_cond = (dist['species'].isin(['Sum']))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = var_col_cond & dir_cond & prop_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # Rescaling bin centers by crowder size

    data['confinement_rate'] = np.round(
        (data['dcrowd'])/
         (data['dcyl'] - data['dcrowd']), 3)
    data['confinement_rate_r'] = np.round(
         (data['dcyl'] - data['dcrowd'])/(data['dcrowd']), 3)
    data['dep_energy_max'] = np.round(
        data['phi_c_bulk_round'] * (1 +
        (3 * 1.0) / (2 * data['dcrowd'])
        ),3)
    data['int_energy_max'] = np.round(
        data['nmon'] * data['phi_c_bulk_round'] * (
        1.0 / data['dcrowd']) * ( 3 * 1.0 * data['dcrowd'] + 3/2), 3)
    # phi_c_rescaled = dmon * phi_c_bulk / dcrowd
    data['phi_c_rescaled'] = np.round(
        1.0 * data['phi_c_bulk'] / data['dcrowd'], 
        3
    )
    # plotting style

    ext = 'pdf'
    save_to = './'

    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'
    
    col_attr = 'phi_c_bulk_round'
    col_order = sorted(data[col_attr].unique())

    style_attr = 'nmon'
    style_order = sorted(data[style_attr].unique())

    size_attr = 'dcyl'
    size_order = sorted(data[size_attr].unique())
    sizes=[3+2*i for i in range(len(size_order))]

    hue_attr = 'dcrowd'
    hue_order = sorted(data[hue_attr].unique())
    color_palette = sns.color_palette("rocket_r", len(hue_order))

    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (1, 0.98),
        'frameon': True,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 3
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery = 4
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        sizes=sizes,
        col=col_attr,
        col_order=col_order,
        col_wrap=3,
        hue=hue_attr,
        hue_order=hue_order,
        markers=True,
        dashes=False ,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=4,
        ci=None,
        )   
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[size_attr]] + list(size_order) + \
                     [attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_ylabels(properties_labels[project][direction+property_+'-norm']['Sum'])
    tseries_grid.set_titles(attr_labels[col_attr] + r"$={col_name}$")
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    tseries_grid.fig.tight_layout(pad=0)
    output = "-".join(
        ["equilPlot", project, direction + property_, col_attr, hue_attr, style_attr, size_attr, 'SumRuleEquationAlone']
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

#### Sum rule alone: a_c=4, 6 ignored

In [ ]:
# Filtering dataset
for direction in ['z', 'r']:
    property_ = 'Phi'
    dcrowd_cond = (dist['dcrowd'].isin([4.0,6.0]))
    var_col_cond = (dist['species'].isin(['Sum']))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = var_col_cond & dir_cond & prop_cond & ~dcrowd_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # Rescaling bin centers by crowder size

    data['confinement_rate'] = np.round(
        (data['dcrowd'])/
         (data['dcyl'] - data['dcrowd']), 3)
    data['confinement_rate_r'] = np.round(
         (data['dcyl'] - data['dcrowd'])/(data['dcrowd']), 3)
    data['dep_energy_max'] = np.round(
        data['phi_c_bulk_round'] * (1 +
        (3 * 1.0) / (2 * data['dcrowd'])
        ),3)
    data['int_energy_max'] = np.round(
        data['nmon'] * data['phi_c_bulk_round'] * (
        1.0 / data['dcrowd']) * ( 3 * 1.0 * data['dcrowd'] + 3/2), 3)
    # phi_c_rescaled = dmon * phi_c_bulk / dcrowd
    data['phi_c_rescaled'] = np.round(
        1.0 * data['phi_c_bulk'] / data['dcrowd'], 
        3
    )
    # plotting style

    ext = 'pdf'
    save_to = './'

    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'
    
    col_attr = 'phi_c_bulk_round'
    col_order = sorted(data[col_attr].unique())

    style_attr = 'nmon'
    style_order = sorted(data[style_attr].unique())

    size_attr = 'dcyl'
    size_order = sorted(data[size_attr].unique())
    sizes=[3+2*i for i in range(len(size_order))]

    hue_attr = 'dcrowd'
    hue_order = sorted(data[hue_attr].unique())
    color_palette = sns.color_palette("rocket_r", len(hue_order))

    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (1, 0.98),
        'frameon': True,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 3
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery = 4
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        sizes=sizes,
        col=col_attr,
        col_order=col_order,
        col_wrap=3,
        hue=hue_attr,
        hue_order=hue_order,
        markers=True,
        dashes=False ,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=4,
        ci=None,
        )   
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[size_attr]] + list(size_order) + \
                     [attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_ylabels(properties_labels[project][direction+property_+'-norm']['Sum'])
    tseries_grid.set_titles(attr_labels[col_attr] + r"$={col_name}$")
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    tseries_grid.fig.tight_layout(pad=0)
    output = "-".join(
        ["equilPlot", project, direction + property_, col_attr, hue_attr, style_attr, size_attr, 'SumRuleEquationAlone-LargeDcrowdIgnored']
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

#### Sum rule alone: a_c=1.0

In [ ]:
# Filtering dataset
for direction in ['z', 'r']:
    property_ = 'Phi'
    dcrowd_cond = (dist['dcrowd'].isin([1.0]))
    var_col_cond = (dist['species'].isin(['Sum']))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = var_col_cond & dir_cond & prop_cond & dcrowd_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # Rescaling bin centers by crowder size

    data['confinement_rate'] = np.round(
        (data['dcrowd'])/
         (data['dcyl'] - data['dcrowd']), 3)
    data['confinement_rate_r'] = np.round(
         (data['dcyl'] - data['dcrowd'])/(data['dcrowd']), 3)
    data['dep_energy_max'] = np.round(
        data['phi_c_bulk_round'] * (1 +
        (3 * 1.0) / (2 * data['dcrowd'])
        ),3)
    data['int_energy_max'] = np.round(
        data['nmon'] * data['phi_c_bulk_round'] * (
        1.0 / data['dcrowd']) * ( 3 * 1.0 * data['dcrowd'] + 3/2), 3)
    # phi_c_rescaled = dmon * phi_c_bulk / dcrowd
    data['phi_c_rescaled'] = np.round(
        1.0 * data['phi_c_bulk'] / data['dcrowd'], 
        3
    )
    # plotting style

    ext = 'pdf'
    save_to = './'

    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'

    col_attr = 'phi_c_bulk_round'
    col_order = sorted(data[col_attr].unique())
    
    style_attr = 'nmon'
    style_order = data[style_attr].unique()
    style_order.sort()

    size_attr = 'nmon'
    size_order = data[size_attr].unique()
    size_order.sort()
    sizes=[4+2*i for i in range(len(size_order))]

    hue_attr = 'dcyl'
    hue_order = data[hue_attr].unique()
    hue_order = sorted(hue_order)
    color_palette = sns.color_palette("rocket_r", len(hue_order))

    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (1, 0.98),
        'frameon': True,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 3
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery = 4
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        sizes=sizes,
        col=col_attr,
        col_order=col_order,
        col_wrap=3,
        hue=hue_attr,
        hue_order=hue_order,
        markers=True,
        dashes=False ,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=4,
        ci=None,
        )   
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[size_attr]] + list(size_order) #+ \
                     #[attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_ylabels(properties_labels[project][direction+property_+'-norm']['Sum'])
    tseries_grid.set_titles(attr_labels[col_attr] + r"$={col_name}$")
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    tseries_grid.fig.tight_layout(pad=0.35)
    output = "-".join(
        ["equilPlot", project, direction + property_, col_attr, hue_attr, style_attr, size_attr, 'SumRuleEquationAlone-DcrowdEqualDmon']
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

#### Sum rule with other local distributions

In [ ]:
# Filtering dataset
for direction in ['z','r']:
    property_ = 'Phi'
    phi_cond = (dist['phi_c_bulk_round'].isin([0.1,0.2,0.3,0.4]))
    dcrowd_cond = (dist['dcrowd'].isin([0.6, 1.0, 2.0]))
    dcyl_cond = (dist['dcyl'].isin([10, 15, 20]))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = phi_cond & dir_cond & prop_cond & dcrowd_cond & dcyl_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # Rescaling bin centers by crowder size

    data['confinement_rate'] = np.round(
        (data['dcrowd'])/
         (data['dcyl'] - data['dcrowd']), 3)
    data['confinement_rate_r'] = np.round(
         (data['dcyl'] - data['dcrowd'])/(data['dcrowd']), 3)
    data['dep_energy_max'] = np.round(
        data['phi_c_bulk_round'] * (1 +
        (3 * 1.0) / (2 * data['dcrowd'])
        ),3)
    data['int_energy_max'] = np.round(
        data['nmon'] * data['phi_c_bulk_round'] * (
        1.0 / data['dcrowd']) * ( 3 * 1.0 * data['dcrowd'] + 3/2), 3)
    # phi_c_rescaled = dmon * phi_c_bulk / dcrowd
    data['phi_c_rescaled'] = np.round(
        1.0 * data['phi_c_bulk'] / data['dcrowd'], 
        3
    )
    # plotting style

    ext = 'pdf'
    save_to = './'

    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'
    
    col_attr = 'dcyl'
    col_order = sorted(data[col_attr].unique())
    
    row_attr = 'phi_c_bulk_round'
    row_order = sorted(data[row_attr].unique())

    style_attr = 'species'
    style_order = sorted(data[style_attr].unique())

    size_attr = 'nmon'
    size_order = sorted(data[size_attr].unique())
    sizes=[3+2*i for i in range(len(size_order))]

    hue_attr = 'dcrowd'
    hue_order = sorted(data[hue_attr].unique())
    color_palette = sns.color_palette("rocket_r", len(hue_order))
      
    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (0.87, 0.96),
        'frameon': True,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 4
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery == 5
    else: 
        markevery = 30
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        markers=False,
        dashes=[(2,2), (1,1), (1,0)],
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=6,
        ci=None,
        )   
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.legendHandles[idx].set_linewidth(8)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    for ax in tseries_grid.axes.flatten():
        ax.set_ylabel(
            properties_labels[project][direction+property_+'-norm']['symbol']
        )
        ax2 = ax.twinx()
        ax2.set_ylim(ax.get_ylim())
        #ax2.set_yticks(ax.get_yticks())
        ax2.set_ylabel(
            properties_labels[project][direction+property_+'-norm']['Sum']
        )
    #tseries_grid.figure.text(0.9, 0.74, r"Increase in $\phi_c:\downarrow$")
    #tseries_grid.figure.text(0.9, 0.71, r"Increase in $D:\rightarrow$")
    tseries_grid.set_titles(attr_labels[row_attr] + "$={row_name}$ | " + attr_labels[col_attr] + r"$={col_name}$")
    
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    tseries_grid.tight_layout(w_pad=0.5)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(
        ["equilPlot", project, direction + property_, col_attr, row_attr, hue_attr, style_attr, size_attr, 'NormalizedScaled']
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

#### Sumrule with other local dist: species per row

In [ ]:
# Filtering dataset
species_shortnames = {
    'Crowders': 'Crd',
    'Chain': 'Mon',
    'Sum': 'Sum'
}

for direction in ['z','r']:
    property_ = 'Phi'
    phi_cond = (dist['phi_c_bulk_round'].isin([0.1,0.2,0.3,0.4]))
    #dcrowd_cond = (dist['dcrowd'].isin([0.6, 1.0, 2.0]))
    #dcyl_cond = (dist['dcyl'].isin([10, 15, 20]))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = phi_cond & dir_cond & prop_cond #& dcrowd_cond & dcyl_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # Rescaling bin centers by crowder size

    data['confinement_rate'] = np.round(
        (data['dcrowd'])/
         (data['dcyl'] - data['dcrowd']), 3)
    data['confinement_rate_r'] = np.round(
         (data['dcyl'] - data['dcrowd'])/(data['dcrowd']), 3)
    data['dep_energy_max'] = np.round(
        data['phi_c_bulk_round'] * (1 +
        (3 * 1.0) / (2 * data['dcrowd'])
        ),3)
    data['int_energy_max'] = np.round(
        data['nmon'] * data['phi_c_bulk_round'] * (
        1.0 / data['dcrowd']) * ( 3 * 1.0 * data['dcrowd'] + 3/2), 3)
    # phi_c_rescaled = dmon * phi_c_bulk / dcrowd
    data['phi_c_rescaled'] = np.round(
        1.0 * data['phi_c_bulk'] / data['dcrowd'], 
        3
    )
    # plotting style

    ext = 'pdf'
    save_to = './'

    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'
    
    col_attr = 'phi_c_bulk_round'
    col_order = sorted(data[col_attr].unique())
    
    row_attr = 'species'
    row_order = ['Crowders', 'Chain', 'Sum']

    style_attr = 'dcyl'
    style_order = sorted(data[style_attr].unique())

    size_attr = 'nmon'
    size_order = sorted(data[size_attr].unique())
    sizes=[3+2*i for i in range(len(size_order))]

    hue_attr = 'dcrowd'
    hue_order = sorted(data[hue_attr].unique())
    color_palette = sns.color_palette("rocket_r", len(hue_order))
      
    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (0.92, 0.96),
        'frameon': False,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 4
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery == 5
    else: 
        markevery = 30
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        sizes=sizes,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        markers=True,
        #dashes=[(2,2), (1,1), (1,0)],
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=6,
        ci=None,
        )  
    for idx, ax in enumerate(tseries_grid.axes[-1,:]):
        ax.axhline(
            1.0,
            color='darkgreen',
            lw=5,
            label=properties_labels[project][direction+property_+'-norm']['Sum_constant']
        )
        ax.legend(frameon=True,
                  #bbox_to_anchor= (0.12, 0.96),
                  facecolor= 'white')
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[size_attr]] + list(size_order) + \
                     [attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        #tseries_grid._legend.legendHandles[idx].set_linewidth(8)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    for species, ax in zip(row_order, tseries_grid.axes[:,0]):
        ax.set_ylabel(
            properties_labels[project][direction+property_+'-norm'][species_shortnames[species]]
        )  
    #tseries_grid.figure.text(0.9, 0.74, r"Increase in $\phi_c:\downarrow$")
    #tseries_grid.figure.text(0.9, 0.71, r"Increase in $D:\rightarrow$")
    tseries_grid.set_titles("{row_name} | " + attr_labels[col_attr] + r"$={col_name}$")
    #tseries_grid.figure.text(0.9, 0.74, r"Increase in $\phi_c:\downarrow$")
    #tseries_grid.figure.text(0.9, 0.71, r"Increase in $D:\rightarrow$")
    tseries_grid.set_titles("{row_name} | " + attr_labels[col_attr] + r"$={col_name}$")
    
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    tseries_grid.tight_layout(w_pad=0)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(
        ["equilPlot", project, direction + property_, 'NormalizedScaled-speciesPerRow']
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

#### Picks and Troughs in radial distributions

##### Phase separartion

In [ ]:
# Filtering dataset
phi_cond = (dist['phi_c_bulk_round'].isin([0.2]))
nmon_cond = (dist['nmon'].isin([500]))
dcrowd_cond = (dist['dcrowd'].isin([1.0]))
dcyl_cond = (dist['dcyl'].isin([10.0]))
var_col_cond = (dist['species'].isin(['Chain', 'Crowders']))
prop_cond = (dist['property'].isin(['Phi', 'Rho']))
dir_cond = (dist['direction'].isin(['r', 'z']))
cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond & dir_cond & prop_cond & dcyl_cond
data = dist.loc[cond, :].copy()
data.sort_values(by=['dcyl','dcrowd'], inplace=True)
data.loc[data['direction']=='z','bin_center-dcrowd-recentered'] = \
    data.loc[data['direction']=='z','bin_center-norm']
#data.loc[(data['direction']=='r') & (data['property']=='Rho'),'norm_species_value'] = \
#    data.loc[(data['direction']=='r') & (data['property']=='Rho'),'scale-normalized_curve_species_value']
data['columns'] = data.apply(
lambda x: rf"$N={x['nmon']}$ - ${{D}}/{{a}}={x['dcyl']}$ - ${{a_c}}/{{a}}={x['dcrowd']}$", axis=1)
# plotting style
ext = 'pdf'
save_to = './'

x_prop = 'bin_center-dcrowd-recentered'
# Dropping  values with bin_center-dcrowd-recentered<0
if x_prop == 'bin_center-dcrowd-recentered':
    data = data.loc[
        data['bin_center-dcrowd-recentered']>=0, :]
y_prop = 'norm_species_value'

data.reset_index(inplace=True, drop=True)

style_attr = 'species'
style_order = data[style_attr].unique()
style_order.sort()

hue_attr = 'phi_c_bulk_round'
hue_order = data[hue_attr].unique()
hue_order.sort()

col_attr = 'direction'
col_order = data[col_attr].unique()
col_order.sort()
row_attr = 'property'
row_order = data[row_attr].unique()
row_order.sort()
color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'grid.color': 'gray',
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
save_to = './'
font_scale = 2
#fig_title_kws = {'x': 0.5, 'y': 0.99, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.85, 0.95),
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'markerscale': 3
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    style=style_attr,
    style_order=style_order,
    col=col_attr,
    col_order=col_order,
    #col_wrap=2,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    markers=False,
    dashes=[(1,0),(2,1)],
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=0.7,
    lw=4,
    ci=None,
    )
legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                 [attr_labels[style_attr]] + list(style_order)           
                )
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
xlims = data.groupby(
        ['property', 'direction', 'columns', 'dcyl', 'dcrowd'])[[x_prop, y_prop, 'bin_center']].max()
xlims.reset_index(inplace=True)
xlims.drop(columns='bin_center', inplace=True)
xlims = xlims.values
for ax, (prop, direction, title, dcyl, dcrowd, x_max, y_max) in zip(tseries_grid.axes.flatten(), xlims):
    ax.set_title(title)
    ax.set_ylabel(properties_labels[project][direction+prop+'-norm']['symbol'])
    ax.set_xlabel(attr_labels[x_prop+'-'+direction])
    if direction == 'r':
        xlim_extra = (dcyl-(dcyl//dcrowd)*dcrowd)
        ax.set_xticks(np.arange(0, (dcyl//dcrowd)+1, 2), minor=False)
        ax.grid(
            axis='y', which='major', color='black', linestyle='-',lw=1
        )
        ax.set_xticks(np.arange(0, (dcyl//dcrowd)+0.25, 1), minor=True)
        ax.grid(
            axis='x', which='major', color='black', linestyle='-',lw=1
        )
        ax.grid(
            axis='x', which='minor', color='gray', linestyle='-',lw=0.8
        )
        colors = sns.color_palette('magma_r',3)
        x_range = np.linspace(0, 0.3*x_max,100)
        line_reg = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[0], alpha=0.4, zorder=0
        )
        x_range = np.linspace(0.3*x_max, 0.52*x_max,100)
        line_layering = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[1], alpha=0.4, zorder=0
        )
        x_range = np.linspace(0.52*x_max, x_max,100)
        line_layering_strong = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[2], alpha=0.4, zorder=0
        )
        if prop == 'Phi':
            ax.legend([line_reg, line_layering, line_layering_strong],
                      ["No wall-layering", "Wall-layering", "Strong wall-layering"],
                      frameon=True,
                      facecolor='white',
                      framealpha=1,
                      loc='lower left'
                 )
    else:
        colors = sns.color_palette('mako_r',3)
        ax.grid(
            axis='x', which='major', color='black', linestyle='-',lw=1
        )
        ax.grid(
            axis='y', which='major', color='black', linestyle='-',lw=1
        )
        x_range = np.linspace(0, 0.36*x_max,100)
        line_chain = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[0], alpha=0.4, zorder=0
        )
        x_range = np.linspace(0.36*x_max, 0.7*x_max,100)
        line_bound = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[1], alpha=0.4, zorder=0
        )
        x_range = np.linspace(0.7*x_max, x_max,100)
        line_crowd = ax.fill_between(
            x_range, 1.02*y_max, -0.02,color=colors[2], alpha=0.4, zorder=0
        )
        if prop == 'Phi':
            ax.legend([line_chain, line_bound, line_crowd],
                      ["Chain-occupying", "Boundary", "Crowder-rich"],
                      framealpha=1.0,
                      frameon=True,
                      facecolor='white'
                 )
        
output = "-".join(
    ["equilPlot", project, 'radialUndulatoryBehavior-phaseSeparation']
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

##### D as rows, phic as hue

In [ ]:
# Filtering dataset
phi_cond = (dist['phi_c_bulk_round'].isin([0.1, 0.2, 0.3, 0.4]))
nmon_cond = (dist['nmon'].isin([500, 2000]))
dcrowd_cond = (dist['dcrowd'].isin([0.6, 2.0]))
dcyl_cond = (dist['dcyl'].isin([10.0, 20.0]))
var_col_cond = (dist['species'].isin(['Chain', 'Crowders']))
prop_cond = (dist['property'].isin(['Phi']))
dir_cond = (dist['direction'].isin(['r', 'z']))
cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond & dir_cond & prop_cond & dcyl_cond
data = dist.loc[cond, :].copy()
data.sort_values(by=['dcyl','dcrowd'], inplace=True)
data.loc[data['direction']=='z','bin_center-dcrowd-recentered'] = \
    data.loc[data['direction']=='z','bin_center-norm']
data.loc[(data['direction']=='r') & (data['property']=='Rho'),'norm_species_value'] = \
    data.loc[(data['direction']=='r') & (data['property']=='Rho'),'scale-normalized_curve_species_value']
data['columns'] = data.apply(
lambda x: rf"$N={x['nmon']}$ - ${{D}}/{{a}}={x['dcyl']}$ - ${{a_c}}/{{a}}={x['dcrowd']}$", axis=1)
# plotting style
ext = 'pdf'
save_to = './'

x_prop = 'bin_center-dcrowd-recentered'
# Dropping  values with bin_center-dcrowd-recentered<0
if x_prop == 'bin_center-dcrowd-recentered':
    data = data.loc[
        data['bin_center-dcrowd-recentered']>=0, :]
y_prop = 'norm_species_value'

style_attr = 'species'
style_order = data[style_attr].unique()
style_order.sort()

hue_attr = 'phi_c_bulk_round'
hue_order = data[hue_attr].unique()
hue_order.sort()
color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))

col_attr = 'direction'
col_order = data[col_attr].unique()
col_order.sort()
row_attr = 'columns'
row_order = data[row_attr].unique()
row_order.sort()

height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'grid.color': 'gray',
    'axes.grid.axis': 'x',
    #'axes.grid.which': 'both'
    }
save_to = './'
font_scale = 2
#fig_title_kws = {'x': 0.5, 'y': 0.99, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.85, 0.95),
    'frameon': True,
    'facecolor': 'white',
    'framealpha':1.0,
    'markerscale': 3
}
ax_move_legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    #'borderpad': 0.1,
    'framealpha':1.0,
    'markerscale': 3
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    style=style_attr,
    style_order=style_order,
    col=col_attr,
    col_order=col_order,
    #col_wrap=2,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    markers=False,
    dashes=[(1,0),(2,1)],
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=0.7,
    lw=4,
    ci=None,
    )
legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                 #[attr_labels[size_attr]] + list(size_order) + \
                 [attr_labels[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid_handles = tseries_grid._legend.legendHandles
tseries_grid.axes[0,1].legend(
    tseries_grid_handles[:len(list(hue_order))+1],
    legend_labels[:len(list(hue_order))+1],
    **ax_move_legend_kws
)
tseries_grid.axes[1,1].legend(
    tseries_grid_handles[len(list(hue_order))+1:],
    legend_labels[len(list(hue_order))+1:],
    **ax_move_legend_kws
)
xlims = data.groupby(
        [row_attr, col_attr, 'dcyl', 'property', 'dcrowd'])[[x_prop, y_prop, 'bin_center']].max()
xlims.reset_index(inplace=True)
xlims.drop(columns='bin_center', inplace=True)
xlims = xlims.values
for ax, (title, direction, dcyl, prop, dcrowd, x_max, y_max) in zip(tseries_grid.axes.flatten(), xlims):
    ax.set_title(title)
    ax.set_ylabel(properties_labels[project][direction+prop+'-norm']['symbol'])
    ax.set_xlabel(attr_labels[x_prop+'-'+direction])
    if direction == 'r':
        xlim_extra = (dcyl-(dcyl//dcrowd)*dcrowd)
        ax.set_xticks(np.arange(0, (dcyl//dcrowd)+1, 2), minor=False)
        ax.grid(
            axis='y', which='major', color='black', linestyle='-',lw=1
        )
        ax.set_xticks(np.arange(0, (dcyl//dcrowd)+0.25, 1), minor=True)
        ax.grid(
            axis='x', which='major', color='black', linestyle='-',lw=1
        )
        ax.grid(
            axis='x', which='minor', color='gray', linestyle='-',lw=0.8
        )
    else:
        ax.grid(
            axis='x', which='major', color='black', linestyle='-',lw=1
        )
        ax.grid(
            axis='y', which='major', color='black', linestyle='-',lw=1
        )
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.set_titles("{row_name}")
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(
    ["equilPlot", project, 'radialUndulatoryBehavior-phiCrowd']
)
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

##### dcyl as cols, phic as row, dcrowd as hue

In [ ]:
# Filtering dataset
phi_cond = (dist['phi_c_bulk_round'].isin([0.2, 0.3, 0.4]))
nmon_cond = (dist['nmon'].isin([2000]))
dcrowd_cond = (dist['dcrowd'].isin([0.6, 0.8, 1.0, 2.0, 4.0, 6.0]))
dcyl_cond = (dist['dcyl'].isin([25.0]))
var_col_cond = (dist['species'].isin(['Chain', 'Crowders']))
prop_cond = (dist['property'].isin(['Phi']))
dir_cond = (dist['direction'].isin(['r', 'z']))
cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond & dir_cond & prop_cond & dcyl_cond
data = dist.loc[cond, :].copy()
data['columns'] = data.apply(
lambda x: rf"$N={x['nmon']}$ - ${{D}}/{{a}}={x['dcyl']}$ - $\phi_c={x['phi_c_bulk_round']}$", axis=1)
# plotting style
ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm'
y_prop = 'norm_species_value'

style_attr = 'species'
style_order = data[style_attr].unique()
style_order.sort()

hue_attr = 'dcrowd'
hue_order = data[hue_attr].unique()
hue_order.sort()

size_attr = 'dcyl'
size_order = data[size_attr].unique()
size_order = sorted(size_order, reverse=True)

col_attr = 'direction'
col_order = data[col_attr].unique()
col_order.sort()

row_attr = 'phi_c_bulk_round'
row_order = data[row_attr].unique()
row_order.sort()

color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'grid.color': 'black',
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
ax_move_legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'markerscale': 3
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    style=style_attr,
    style_order=style_order,
    col=col_attr,
    col_order=col_order,
    #col_wrap=2,
    row=row_attr,
    row_order=row_order,
    #size=size_attr,
    #size_order=size_order,
    #size_norm=(0.2,0.3),
    hue=hue_attr,
    hue_order=hue_order,
    markers=False,
    dashes=[(1,0),(2,1)],
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=0.7,
    markersize=10,
    markevery=15,
    lw=4,
    ci=None,
    )
legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                 #[attr_labels[size_attr]] + list(size_order) + \
                 [attr_labels[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid_handles = tseries_grid._legend.legendHandles
tseries_grid.axes[0,1].legend(
    tseries_grid_handles[:len(list(hue_order))+1],
    legend_labels[:len(list(hue_order))+1],
    **ax_move_legend_kws
)
tseries_grid.axes[1,1].legend(
    tseries_grid_handles[len(list(hue_order))+1:],
    legend_labels[len(list(hue_order))+1:],
    **ax_move_legend_kws
)
xlims = data.groupby([
    row_attr, col_attr,
    'columns', 'dcyl'])[[x_prop,'bin_center']].max()
xlims.reset_index(inplace=True)
xlims.drop(columns='bin_center', inplace=True)
xlims = xlims.values
for ax, (phi_c_bulk_round, direction, title, dcyl, x_max) in zip(tseries_grid.axes.flatten(), xlims):
    ax.set_title(title)
    ax.set_ylabel(properties_labels[project][direction+'Phi'+'-norm']['symbol'])
    ax.set_xlabel(attr_labels[x_prop+'-'+direction])

output = "-".join(
    ["equilPlot", project, 'radialUndulatoryBehavior-dCrowd']
)
#tseries_grid.fig.set_figwidth(15)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

###### Old approach

In [ ]:
directions = ['r','z']
props = ['Rho', 'Phi']
dir_prop_pairs = list(itertools.product(directions, props))
species_list = ['Mon', 'Crd']
normalizing_methods = ['norm', 'scale-normalized_curve']
# 'norm': normalizing each curve with its deeip inside species-rich region
# 'scale-normalized_curve': normalizing each curve to make the area-under-cruve 1.
normalzing_method = normalizing_methods[1]
melting_dict = [species + '-' + normalzing_method for species in species_list]
melting_dict.append('Sum-sumrule_local')
species_names = ['Chain', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
species_names_dict['Sum'] = 'Sum'
# col to drops for each prop
measures = ['mean','sem', 'var']
ens_avg_measures = [
    species + '-' + col for species in species_list for col in measures
]
# Database location
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'all'
geometry = 'biaxial'
var_cols = []
for direction, prop in dir_prop_pairs:
    prop_fullname = direction + prop
    equil_db = '-'.join(['allInOne', project, group, prop_fullname])
    equil_db += '-NormalizedScaled.parquet.brotli'
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # Dropping redundant columns
    cols_to_drop = [prop_fullname + redund for redund in ens_avg_measures]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # Sumrule at local level
    local_dist[prop_fullname+'Sum-sumrule_local'] = 0
    local_dist[prop_fullname+'Sum-sumrule_constant'] = 0
    for species in species_list:
        local_dist[prop_fullname+'Sum-sumrule_local'] += \
            local_dist[prop_fullname+species+'-scale']
    local_dist[prop_fullname+'Sum-sumrule_local'] = \
        (local_dist[prop_fullname+'Sum-sumrule_local'] /
         local_dist[prop_fullname+'Crd-normalizer'])
    # Melting based local distributions on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes += ['bin_center', 'bin_center-norm', 'bin_center-dcrowd', 'bin_center-dcrowd-recentered', 'bin_center-recentered-norm']
    var_name = 'species'
    var_cols = [prop_fullname + col for col in melting_dict]
    sel_cols = attributes + var_cols
    local_dist_melted = local_dist[sel_cols].melt(
        id_vars=attributes,
        value_vars=var_cols,
        value_name=var_name+'_value',
        var_name=var_name
    )
    # Renaming values
    local_dist_melted.loc[:, var_name] = \
        local_dist_melted.loc[:, var_name].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
        )
    local_dist_melted.loc[:, var_name] = \
        local_dist_melted.loc[:, var_name].apply(
            lambda x: species_names_dict[x]
    )
    # Dropping distributions with phi_c=0
    local_dist_melted = \
        local_dist_melted[local_dist_melted['phi_c_bulk_round']!=0]
    # Fixing near-end problem in crowder distributions in z direction
    if direction == 'z':
        cond = local_dist_melted['bin_center-norm'] < 0.98
        local_dist_melted = local_dist_melted.loc[cond, :]
    
    # plotting style
    ext = 'pdf'
    save_to = './'
    phi_cond = (local_dist_melted['phi_c_bulk_round'].isin([0.3]))
    nmon_cond = (local_dist_melted['nmon'].isin([500, 2000]))
    dcrowd_cond = (local_dist_melted['dcrowd'].isin([0.6, 0.8, 2.0, 4.0]))
    dcyl_cond = (local_dist_melted['dcyl'].isin([10.0, 25.0]))
    var_col_cond = (local_dist_melted[var_name].isin(['Chain', 'Crowders']))
    cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond & dcyl_cond
    local_dist_melted = local_dist_melted.loc[cond, :]
    local_dist_melted.sort_values(by=['dcyl','dcrowd'], inplace=True)
    local_dist_melted['columns'] = local_dist_melted.apply(
    lambda x: rf"$N={x['nmon']}$ - ${{D}}/{{a}}={x['dcyl']}$ - ${{a_c}}/{{a}}={x['dcrowd']}$", axis=1)
    
    data = local_dist_melted
    if direction == 'r':
        x_prop = 'bin_center-norm'
    else:
        x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = var_name + '_value'
    
    style_attr = var_name
    style_order = data[style_attr].unique()
    style_order.sort()
    
    hue_attr = 'dcrowd'
    hue_order = data[hue_attr].unique()
    hue_order.sort()
    
    col_attr = 'dcyl'
    col_order = data[col_attr].unique()
    col_order.sort()
    
    row_attr = 'phi_c_bulk_round'
    row_order = data[row_attr].unique()
    row_order.sort()
    
    #color_palette = 'tab10'
    #color_palette = 'flare_cmap_cut'
    color_palette = sns.color_palette("rocket_r", len(hue_order))
    #color_palette = 'rocket_r'
    
    # Annotation setting
    hue_dict = dict(zip(hue_order,color_palette)) 
    annote_tuning = {
        "Phi":{ # prop
            0.6: 0.018,
            0.8: 0.012,
            1.0: 0.02,
            2.0: 0.022,
            4.0: 0.022,
            6.0: 0.012,
            "ytext": 0.003
        },
        "Rho":{
            0.6: 0.018,
            0.8: 0.012,
            1.0: 0.04,
            2.0: 0.04,
            4.0: 0.04,
            6.0: 0.05,
            "ytext": 0.005
        }
    }
    
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': False,
        'grid.color': 'gray',
        'axes.grid.axis': 'x',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 2
    #fig_title_kws = {'x': 0.5, 'y': 0.99, 'ha': 'center'}
    facet_kws = {'sharey': 'row', 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (0.92, 0.9),
        'frameon': False,
        'facecolor': 'white',
        'borderpad': 0.1,
        'markerscale': 3
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery = 5
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        markers=False,
        dashes=True,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=15,
        markevery=markevery,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[style_attr]] + list(style_order))
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.legendHandles[idx].set_linewidth(7)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_titles(
        r"$N=2000$ | " + attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + 
        r"$={col_name}$"
    )
    if direction == 'r':
        dcyl_dcrowds = data.groupby('dcyl')
        dcyl_dcrowds = dcyl_dcrowds['dcrowd'].unique().reset_index().values
        for col_idx, (dcyl, dcrowds) in enumerate(dcyl_dcrowds):
            for row_idx, ax in enumerate(tseries_grid.axes[:, col_idx]):
                for dcrowd in dcrowds:
                    ax.axvline(
                        1-dcrowd/dcyl,
                        color = hue_dict[dcrowd],
                        linestyle=':',
                        lw=3,
                        zorder=0
                    )
                    ax.axvline(
                        1-3*dcrowd/dcyl,
                        color = hue_dict[dcrowd],
                        linestyle=':',
                        lw=3,
                        zorder=0
                    )
                    ax.axvline(
                        1-5*dcrowd/dcyl,
                        color = hue_dict[dcrowd],
                        linestyle=':',
                        lw=3,
                        zorder=0
                    )
                    ax.axvline(
                        1-7*dcrowd/dcyl,
                        color = hue_dict[dcrowd],
                        linestyle=':',
                        lw=3,
                        zorder=0
                    )
                    
                    if (col_idx == 1) & (row_idx == 0):
                        ax.annotate('',
                                    xy=(
                                        1-3*dcrowd/dcyl,
                                        annote_tuning[prop][dcrowd]
                                    ),
                                    xycoords='data',
                                    xytext=(
                                        1-5*dcrowd/dcyl,
                                        annote_tuning[prop][dcrowd]
                                    ),
                                    textcoords='data',
                                    arrowprops=dict(
                                        arrowstyle="<->",
                                        connectionstyle="arc3",
                                        color=hue_dict[dcrowd])
                                    )
                        ax.text(
                            1-4*dcrowd/dcyl-0.05,
                            annote_tuning[prop][dcrowd]+ \
                                annote_tuning[prop]["ytext"],
                            r"${{a_c}}/{{D}}$"
                        )
    tseries_grid.set(xlim=(0-0.05,1.0+0.05))
    #sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(
        ["equilPlot", project, prop_fullname, col_attr, hue_attr, style_attr, 'radialUndulatoryBehavior-dCrowd', normalzing_method]
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

##### dcrowd as cols, phic as row, dcyl as hue

In [ ]:
# Filtering dataset
phi_cond = (dist['phi_c_bulk_round'].isin([0.2,0.3,0.4]))
nmon_cond = (dist['nmon'].isin([2000]))
dcrowd_cond = (dist['dcrowd'].isin([1.0]))
dcyl_cond = (dist['dcyl'].isin([10.0, 15.0, 20.0, 25.0, 30.0]))
var_col_cond = (dist['species'].isin(['Chain', 'Crowders']))
prop_cond = (dist['property'].isin(['Phi']))
dir_cond = (dist['direction'].isin(['r', 'z']))
cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond & dir_cond & prop_cond & dcyl_cond
data = dist.loc[cond, :].copy()
data['columns'] = data.apply(
lambda x: rf"$N={x['nmon']}$ - ${{a_c}}/{{a}}={x['dcrowd']}$ - $\phi_c={x['phi_c_bulk_round']}$", axis=1)
# plotting style
ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm'
y_prop = 'norm_species_value'

style_attr = 'species'
style_order = data[style_attr].unique()
style_order.sort()

hue_attr = 'dcyl'
hue_order = data[hue_attr].unique()
hue_order.sort()

col_attr = 'direction'
col_order = data[col_attr].unique()
col_order.sort()

row_attr = 'phi_c_bulk_round'
row_order = data[row_attr].unique()
row_order.sort()

color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'grid.color': 'black',
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
save_to = './'
font_scale = 2
#fig_title_kws = {'x': 0.5, 'y': 0.99, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.85, 0.95),
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
#    'borderpad': 0.1,
    'markerscale': 3
}
ax_move_legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'markerscale': 3
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    style=style_attr,
    style_order=style_order,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    markers=False,
    dashes=[(1,0),(2,1)],
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=0.7,
    lw=4,
    ci=None,
    )
legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                 #[attr_labels[size_attr]] + list(size_order) + \
                 [attr_labels[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid_handles = tseries_grid._legend.legendHandles
tseries_grid.axes[0,1].legend(
    tseries_grid_handles[:len(list(hue_order))+1],
    legend_labels[:len(list(hue_order))+1],
    **ax_move_legend_kws
)
tseries_grid.axes[1,1].legend(
    tseries_grid_handles[len(list(hue_order))+1:],
    legend_labels[len(list(hue_order))+1:],
    **ax_move_legend_kws
)
xlims = data.groupby([
    row_attr, col_attr,
    'columns'])[[x_prop,'bin_center']].max()
xlims.reset_index(inplace=True)
xlims.drop(columns='bin_center', inplace=True)
xlims = xlims.values
for ax, (dcyl, direction, title, x_max) in zip(tseries_grid.axes.flatten(), xlims):
    ax.set_title(title)
    ax.set_ylabel(properties_labels[project][direction+'Phi'+'-norm']['symbol'])
    ax.set_xlabel(attr_labels[x_prop+'-'+direction])
output = "-".join(
    ["equilPlot", project, 'radialUndulatoryBehavior-dCyl']
)
#tseries_grid.fig.set_figwidth(15)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

###### Old approach

In [ ]:
directions = ['r','z']
props = ['Rho', 'Phi']
dir_prop_pairs = list(itertools.product(directions, props))
species_list = ['Mon', 'Crd']
normalizing_methods = ['norm', 'scale-normalized_curve']
# 'norm': normalizing each curve with its deeip inside species-rich region
# 'scale-normalized_curve': normalizing each curve to make the area-under-cruve 1.
normalzing_method = normalizing_methods[1]
melting_dict = [species + '-' +  normalzing_method for species in species_list]
melting_dict.append('Sum-sumrule_local')
species_names = ['Chain', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
species_names_dict['Sum'] = 'Sum'
# col to drops for each prop
measures = ['mean','sem', 'var']
ens_avg_measures = [
    species + '-' + col for species in species_list for col in measures
]
# Database location
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'all'
geometry = 'biaxial'
var_cols = []
for direction, prop in dir_prop_pairs:
    prop_fullname = direction + prop
    equil_db = '-'.join(['allInOne', project, group, prop_fullname])
    equil_db += '-NormalizedScaled.parquet.brotli'
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # Dropping redundant columns
    cols_to_drop = [prop_fullname + redund for redund in ens_avg_measures]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # Sumrule at local level
    local_dist[prop_fullname+'Sum-sumrule_local'] = 0
    local_dist[prop_fullname+'Sum-sumrule_constant'] = 0
    for species in species_list:
        local_dist[prop_fullname+'Sum-sumrule_local'] += \
            local_dist[prop_fullname+species+'-scale']
    local_dist[prop_fullname+'Sum-sumrule_local'] = \
        (local_dist[prop_fullname+'Sum-sumrule_local'] /
         local_dist[prop_fullname+'Crd-normalizer'])
    # Melting based local distributions on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes += ['bin_center', 'bin_center-norm', 'bin_center-dcrowd', 'bin_center-dcrowd-recentered', 'bin_center-recentered-norm']
    var_name = 'species'
    var_cols = [prop_fullname + col for col in melting_dict]
    sel_cols = attributes + var_cols
    local_dist_melted = local_dist[sel_cols].melt(
        id_vars=attributes,
        value_vars=var_cols,
        value_name=var_name+'_value',
        var_name=var_name
    )
    # Renaming values
    local_dist_melted.loc[:, var_name] = \
        local_dist_melted.loc[:, var_name].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
        )
    local_dist_melted.loc[:, var_name] = \
        local_dist_melted.loc[:, var_name].apply(
            lambda x: species_names_dict[x]
    )
    # Dropping distributions with phi_c=0
    local_dist_melted = \
        local_dist_melted[local_dist_melted['phi_c_bulk_round']!=0]
    # Fixing near-end problem in crowder distributions in z direction
    if direction == 'z':
        cond = local_dist_melted['bin_center-norm'] < 0.98
        local_dist_melted = local_dist_melted.loc[cond, :]
    
    # plotting style
    ext = 'pdf'
    save_to = './'
    phi_cond = (local_dist_melted['phi_c_bulk_round'].isin([0.2, 0.3]))
    nmon_cond = (local_dist_melted['nmon'].isin([2000]))
    dcrowd_cond = (local_dist_melted['dcrowd'].isin([2.0]))
    var_col_cond = (local_dist_melted[var_name].isin(['Chain', 'Crowders']))
    cond = phi_cond & nmon_cond & dcrowd_cond & var_col_cond
    local_dist_melted = local_dist_melted.loc[cond, :]
    local_dist_melted.sort_values(by=['dcyl','dcrowd'], inplace=True)
    local_dist_melted['columns'] = local_dist_melted.apply(
    lambda x: rf"$N={x['nmon']}$ - ${{D}}/{{a}}={x['dcyl']}$ - ${{a_c}}/{{a}}={x['dcrowd']}$", axis=1)
    
    data = local_dist_melted
    if direction == 'r':
        x_prop = 'bin_center-norm'
    else:
        x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = var_name + '_value'
    
    style_attr = var_name
    style_order = data[style_attr].unique()
    style_order.sort()
    
    hue_attr = 'dcyl'
    hue_order = data[hue_attr].unique()
    hue_order.sort()
    
    col_attr = 'dcrowd'
    col_order = data[col_attr].unique()
    col_order.sort()
    
    row_attr = 'phi_c_bulk_round'
    row_order = data[row_attr].unique()
    row_order.sort()
    
    #color_palette = 'tab10'
    #color_palette = 'flare_cmap_cut'
    color_palette = sns.color_palette("rocket_r", len(hue_order))
    #color_palette = 'rocket_r'
    
    # Annotation setting
    hue_dict = dict(zip(hue_order,color_palette)) 
    annote_tuning = {
        "Phi":{ # prop
            20.0: 0.005,
            25.0: 0.0005,
            30.0: 0.0005,
            "ytext": 0.001
        },
        "Rho":{
            20.0: 0.031,
            25.0: 0.025,
            30.0: 0.035,
            "ytext": 0.002
        }
    }  
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': False,
        'grid.color': 'gray',
        'axes.grid.axis': 'x',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 2
    #fig_title_kws = {'x': 0.5, 'y': 0.99, 'ha': 'center'}
    facet_kws = {'sharey': 'row', 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (0.75, 0.75),
        'frameon': False,
        'facecolor': 'white',
        'borderpad': 0.1,
        'markerscale': 3
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery = 5
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        markers=False,
        dashes=True,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=15,
        markevery=markevery,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[style_attr]] + list(style_order))
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.legendHandles[idx].set_linewidth(7)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    tseries_grid.set_titles(
        r"$N=2000$ | " + attr_labels[row_attr] + r"$={row_name}$ | " + attr_labels[col_attr] + 
        r"$={col_name}$"
    )
    if direction == 'r':
        dcrowd_dcyls = data.groupby('dcrowd')
        dcrowd_dcyls = dcrowd_dcyls['dcyl'].unique().reset_index().values
        for col_idx, (dcrowd, dcyls) in enumerate(dcrowd_dcyls):
            for row_idx, ax in enumerate(tseries_grid.axes[:, col_idx]):
                for dcyl in dcyls:
                    ax.axvline(
                        1-1*dcrowd/dcyl,
                        color = hue_dict[dcyl],
                        linestyle=':',
                        lw=3
                    )
                    ax.axvline(
                        1-3*dcrowd/dcyl,
                        color = hue_dict[dcyl],
                        linestyle=':',
                        lw=3
                    )
                    ax.axvline(
                        1-5*dcrowd/dcyl,
                        color = hue_dict[dcyl],
                        linestyle=':',
                        lw=3
                    )
                    ax.axvline(
                        1-7*dcrowd/dcyl,
                        color = hue_dict[dcyl],
                        linestyle=':',
                        lw=3
                    )
                    ax.axvline(
                        1-9*dcrowd/dcyl,
                        color = hue_dict[dcyl],
                        linestyle=':',
                        lw=3
                    )
                    if (col_idx == 0) & (row_idx == 1):
                        ax.annotate('',
                                    xy=(
                                        1-5*dcrowd/dcyl,
                                        annote_tuning[prop][dcyl]
                                    ),
                                    xycoords='data',
                                    xytext=(
                                        1-7*dcrowd/dcyl,
                                        annote_tuning[prop][dcyl]
                                    ),
                                    textcoords='data',
                                    arrowprops=dict(
                                        arrowstyle="<->",
                                        connectionstyle="arc3",
                                        color=hue_dict[dcyl]
                                    )
                                    )
                        ax.text(
                            1-6*dcrowd/dcyl-0.06,
                            annote_tuning[prop][dcyl]+\
                                annote_tuning[prop]["ytext"],
                            r"${{a_c}}/{{D}}$"
                        )
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(
        ["equilPlot", project, prop_fullname, col_attr, hue_attr, style_attr, 'radialUndulatoryBehavior-dCyl', normalzing_method]
    )
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

##### a_c=a

In [ ]:
# Filtering dataset
phi_cond = (dist['phi_c_bulk_round'].isin([0.2,0.3,0.4]))
#nmon_cond = (dist['nmon'].isin([500]))
dcrowd_cond = (dist['dcrowd'].isin([1.0]))
#dcyl_cond = (dist['dcyl'].isin([10.0, 25.0]))
var_col_cond = (dist['species'].isin(['Chain', 'Crowders']))
prop_cond = (dist['property'].isin(['Phi']))
dir_cond = (dist['direction'].isin(['r', 'z']))
cond = phi_cond & dcrowd_cond & var_col_cond & dir_cond & prop_cond
data = dist.loc[cond, :].copy()
data['columns'] = data.apply(
lambda x: rf"$N={x['nmon']}$ - $D={x['dcyl']}$", axis=1)
data.sort_values(by=['dcyl','nmon'], inplace=True)
# plotting style
ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm'
y_prop = 'norm_species_value'

style_attr = 'species'
style_order = data[style_attr].unique()
style_order.sort()

hue_attr = 'phi_c_bulk_round'
hue_order = data[hue_attr].unique()
hue_order.sort()

col_attr = 'direction'
col_order = data[col_attr].unique()
col_order.sort()

row_attr = 'columns'
row_order = data[row_attr].unique()
#row_order.sort()

color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'grid.color': 'black',
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
ax_move_legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1,
    'markerscale': 3
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    style=style_attr,
    style_order=style_order,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    markers=False,
    dashes=[(1,0),(2,1)],
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=0.7,
    markersize=10,
    markevery=15,
    lw=4,
    ci=None,
    )
legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                 #[attr_labels[size_attr]] + list(size_order) + \
                 [attr_labels[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid_handles = tseries_grid._legend.legendHandles
tseries_grid.axes[0,1].legend(
    tseries_grid_handles[:len(list(hue_order))+1],
    legend_labels[:len(list(hue_order))+1],
    **ax_move_legend_kws
)
tseries_grid.axes[1,1].legend(
    tseries_grid_handles[len(list(hue_order))+1:],
    legend_labels[len(list(hue_order))+1:],
    **ax_move_legend_kws
)
tseries_grid.set_titles("{row_name}")
xlims = data.groupby([row_attr, col_attr])[[x_prop,'bin_center']].max()
xlims.reset_index(inplace=True)
xlims.drop(columns='bin_center', inplace=True)
xlims = xlims.values
for ax, (phi_c_bulk_round, direction, x_max) in zip(tseries_grid.axes.flatten(), xlims):
    ax.set_ylabel(properties_labels[project][direction+'Phi'+'-norm']['symbol'])
    ax.set_xlabel(attr_labels[x_prop+'-'+direction])

output = "-".join(
    ["equilPlot", project, 'radialUndulatoryBehavior-ac_equal_a']
)
#tseries_grid.fig.set_figwidth(15)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

### Trans-Foci

##### Meging different distributions along different directions

In [ ]:
directions = ['r','z']
props = ['Phi', 'Rho']
dir_prop_pairs = list(itertools.product(directions, props))
species_list = ['Foci', 'Mon', 'Crd']
normalizing_methods = ['norm', 'scale-normalized_curve']
# 'norm': normalizing each curve with its deeip inside species-rich region
# 'scale-normalized_curve': normalizing each curve to make the area-under-cruve 1.
species_names = ['Big mon.', 'Small mon.', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
species_names_dict['Sum'] = 'Sum'
# col to drops for each prop
measures = ['mean','sem', 'var']
ens_avg_measures = [
    species + '-' + col for species in species_list for col in measures
]
# Database location
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
#allInOne_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets/'
group = 'all'
geometry = 'biaxial'
var_cols = []
dist = []
for normalizing_method in normalizing_methods:
    dist_per_norm = []
    melting_dict = [species + '-' + normalizing_method for species in species_list]
    melting_dict.append('Sum-sumrule_local')
    for direction, prop in dir_prop_pairs:
        prop_fullname = direction + prop
        equil_db = '-'.join(['allInOne', project, group, prop_fullname])
        equil_db += '-NormalizedScaled.parquet.brotli'
        equil_db = allInOne_db + equil_db
        local_dist = pd.read_parquet(equil_db)
        # Dropping redundant columns
        cols_to_drop = [prop_fullname + redund for redund in ens_avg_measures]
        local_dist.drop(columns=cols_to_drop, inplace=True)
        # Sumrule at local level
        local_dist[prop_fullname+'Sum-sumrule_local'] = 0
        local_dist[prop_fullname+'Sum-sumrule_constant'] = 0
        for species in species_list:
            local_dist[prop_fullname+'Sum-sumrule_local'] += \
                local_dist[prop_fullname+species+'-scale']
        local_dist[prop_fullname+'Sum-sumrule_local'] = \
            (local_dist[prop_fullname+'Sum-sumrule_local'] /
             local_dist[prop_fullname+'Crd-normalizer'])
        # Melting based local distributions on species:
        attributes = project_details[project]['equil_attributes'].copy()
        attributes += ['bin_center', 'bin_center-norm', 'bin_center-dcrowd', 'bin_center-dcrowd-recentered']
        var_name =  'species'
        var_cols = [prop_fullname + col for col in melting_dict]
        sel_cols = attributes + var_cols
        local_dist_melted = local_dist[sel_cols].melt(
            id_vars=attributes,
            value_vars=var_cols,
            value_name=normalizing_method+'_'+var_name+'_value',
            var_name=var_name
        )
        # Renaming values
        local_dist_melted.loc[:, var_name] = \
            local_dist_melted.loc[:, var_name].apply(
                lambda x: x.split('-')[0].split(prop_fullname)[-1]
            )
        local_dist_melted.loc[:, var_name] = \
            local_dist_melted.loc[:, var_name].apply(
                lambda x: species_names_dict[x]
        )
        # Dropping distributions with phi_c=0
        local_dist_melted = \
            local_dist_melted[local_dist_melted['phi_c_bulk_round']!=0]
        # Fixing near-end problem in crowder distributions in z directio
        if direction == 'z':
            cond = local_dist_melted['bin_center-norm'] <= 0.5
            local_dist_melted = local_dist_melted.loc[cond, :]
        # local_dist_melted
        local_dist_melted['direction'] = direction
        local_dist_melted['property'] = prop
        dist_per_norm.append(local_dist_melted)
    dist_per_norm = pd.concat(dist_per_norm, axis=0)
    dist_per_norm.reset_index(inplace=True, drop=True)
    dist_per_norm.drop(columns=['ensemble', 'space', 'ensemble_long'], inplace=True)
    dist.append(dist_per_norm)
dist = pd.concat(dist, axis=1)
dist.reset_index(inplace=True, drop=True)
dist = dist.loc[:, ~dist.columns.duplicated()].copy()

spaces = dist[['nmon_small','nmon_large', 'dcyl', 'dcrowd']].drop_duplicates().sort_values(by=['nmon_small','nmon_large', 'dcyl', 'dcrowd']).values
project_titles = {}
for (nmon_small, nmon_large, dcyl, dcrowd) in spaces:
    project_title_style = {
        'TransFoci': fr" $N_m={nmon_small}, N_M={nmon_large}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
    
del directions, props, dir_prop_pairs, species_list, normalizing_methods
del normalizing_method, melting_dict, species_names, species_names_dict
del measures, ens_avg_measures, allInOne_db, group, geometry, var_cols, prop
del direction, local_dist, local_dist_melted, var_name,  attributes, sel_cols

##### Sumrule with other local dist: species per row

In [ ]:
# Filtering dataset
species_shortnames = {
    'Crowders': 'Crd',
    'Small mon.': 'Mon',
    'Big mon.': 'Foci', 
    'Sum': 'Sum'
}
for direction in ['z','r']:
    property_ = 'Phi'
    #phi_cond = (dist['phi_c_bulk_round'].isin([0.1,0.2,0.3,0.4]))
    #dcrowd_cond = (dist['dcrowd'].isin([0.6, 1.0, 2.0]))
    #dcyl_cond = (dist['dcyl'].isin([10, 15, 20]))
    prop_cond = (dist['property'].isin([property_]))
    dir_cond = (dist['direction'].isin([direction]))
    cond = dir_cond & prop_cond #& phi_cond & dcrowd_cond & dcyl_cond
    data = dist.loc[cond, :].copy()
    data.reset_index(inplace=True, drop=True)
    # plotting style
    # Rescaling bin centers by crowder size
    
    ext = 'pdf'
    save_to = './'
    
    x_prop = 'bin_center-norm'
    x_prop_name = x_prop + '-' + direction
    y_prop = 'norm_species_value'
    
    col_attr = 'dmon_large'
    col_order = sorted(data[col_attr].unique())
    
    row_attr = 'species'
    row_order = ['Crowders', 'Small mon.', 'Big mon.', 'Sum']

    style_attr = 'species'
    style_order = ['Crowders', 'Small mon.', 'Big mon.', 'Sum']

    hue_attr = 'phi_c_bulk_round'
    hue_order = sorted(data[hue_attr].unique())
    color_palette = sns.color_palette("rocket_r", len(hue_order))
    
    height = 9
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
        'axes.facecolor': 'aliceblue',
        'mathtext.default': 'regular',
        'text.usetex': True,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        #'axes.grid.which': 'both'
        }
    save_to = './'
    font_scale = 3
    fig_title_kws = {'x': 0.5, 'y': 0.97, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
    move_legend_kws = {
        'ncol': 1,
        'bbox_to_anchor': (0.86, 0.96),
        'frameon': False,
        'facecolor': 'aliceblue',
        'borderpad': 0.1,
        'markerscale': 4
    }
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    if direction == 'r':
        markevery == 5
    else: 
        markevery = 15
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        markers=True,
        dashes=False,
        #dashes=[(2,2), (1,1), (1,0)],
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        markersize=20,
        markevery=markevery,
        alpha=0.7,
        lw=6,
        ci=None,
        )  
    for idx, ax in enumerate(tseries_grid.axes[-1,:]):
        ax.axhline(
            1.0,
            color='darkgreen',
            lw=5,
            label=properties_labels[project][direction+property_+'-norm']['Sum_constant']
        )
        ax.legend(frameon=True,
                  #bbox_to_anchor= (0.12, 0.96),
                  facecolor= 'white')
    legend_labels = ([attr_labels[hue_attr]] + list(hue_order) + \
                     [attr_labels[style_attr]] + list(style_order)
                    )
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.legendHandles[idx].set_linewidth(8)
        tseries_grid._legend.texts[idx].set_ha('left')
        tseries_grid._legend.texts[idx].set_text(new_label)

    tseries_grid.set_xlabels(attr_labels[x_prop_name])
    for species, ax in zip(row_order, tseries_grid.axes[:,0]):
        ax.set_ylabel(
            properties_labels[project][direction+property_+'-norm'][species_shortnames[species]]
        )  
    #tseries_grid.figure.text(0.9, 0.74, r"Increase in $\phi_c:\downarrow$")
    #tseries_grid.figure.text(0.9, 0.71, r"Increase in $D:\rightarrow$")
    tseries_grid.set_titles("{row_name} | " + attr_labels[col_attr] + r"$={col_name}$")
    #tseries_grid.figure.text(0.9, 0.74, r"Increase in $\phi_c:\downarrow$")
    #tseries_grid.figure.text(0.9, 0.71, r"Increase in $D:\rightarrow$")
    tseries_grid.set_titles("{row_name} | " + attr_labels[col_attr] + r"$={col_name}$")
    
    if direction == 'z':
        tseries_grid.set(xlim=(0-0.02,0.5+0.02),xticks=np.arange(0,0.6,0.1))
    else:
        tseries_grid.set(xlim=(0-0.02,1.0+0.02),xticks=np.arange(0,1.2,0.2))
    tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(
        ["equilPlot", project, direction + property_, 'NormalizedScaled-speciesPerRow']
    )
    tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()